In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings

from get_data_mapping import get_datamapping_pred_vs_last_year_demand, get_data_mapping_summary_product_sales, \
    get_data_mapping_accuracy_segmentation, get_data_mapping_confusion_matrix, get_data_mapping_sku_demand_page, \
    get_data_mapping_sku_demand_page_1
from pred_vs_last_year_demand import generate_prediction_vs_last_year_demand, generate_summary_product_sales, \
    get_accuracy_segmentation_schema, get_forecast_accuracy_schema, get_confusion_matrix_schema, \
    get_sku_demand_page_schema, get_confusion_matrix_schema_new, get_sku_demand_page_schema_1, \
    generate_variability_index, generate_variability_schema
# from schemaPipeline import input_data_c
from utility import make_input_data_continous, collect_data

warnings.filterwarnings('ignore')

output_folder_ = './DATA/RESULTS/'
mape_file_name = 'OCT_24_MAR_25_SCHEMA_DATA_17_10'
# mape_file_name='MAPE_ARIMA_SES_EXP_REXP'
# sheet_name="APR_24"
sheet_name = "Schema_"

writer = pd.ExcelWriter(output_folder_ + mape_file_name + '.xlsx', engine='xlsxwriter')

# SCHEMA 1
final_forecast_path = 'DATA/MAY_24_FORECAST/combined_file_new_oct_24_mar_25.csv'
# forecast_history_path = '../SCHEMA_CODE/DATA/FORECAST_HISTORY/forecast_history.csv'
# forecast_history_path = './DATA/FORECAST_HISTORY/forecast_history.csv'
# forecast_history_path = './DATA/FORECAST_HISTORY/forecast_history_dec_23.csv'
forecast_history_path = './DATA/FORECAST_HISTORY/forecast_history_jul_sep.csv'
input_data_path = './DATA/INPUT_DATA/sales_groupby_31_09_24_final.csv'
final_forecast_data = pd.read_csv(final_forecast_path)
c = final_forecast_data['final_perdiction'].isna() == False
final_forecast_data = final_forecast_data[c]
forecast_history_data = pd.read_csv(forecast_history_path)
input_data = pd.read_csv(input_data_path)
print(final_forecast_data.shape)
print(forecast_history_data.shape)
print(input_data.shape)

categories_present=['Dairy', 'Bakery', 'Fruits', 'Beverage',
                    'Cheese', 'Chocolates', 'Fat & Oil', 'Filling & Jam',
                    'Grocery', 'Meat', 'Non Food', 'Flour, Grain & Flakes',
                    'Nuts, Seeds & Beans', 'PHD', 'Seafood']

import os

product_hierarchy_path = './DATA/Product_MASTER/'
product_hierarchy_files = os.listdir(product_hierarchy_path)
print(product_hierarchy_files)
col = ['Product', "NetWeight", 'H1_description', "H2_description",
       "H3_description", "H4_description", "H5_description", 'CrossPlantStatus', "LeadTime"
    , "ServiceLevel", 'MrpPurchaser', 'PdProductDescription', "ProductWeightUnit"]
sku_details = collect_data(product_hierarchy_path, product_hierarchy_files, col=col)
# master.rename(columns={'Product':"sku","CrossPlantStatus":"MSTAE"},inplace=True)
sku_details.rename(columns={'Product': "sku", 'H1_description': "H1", "H2_description": "H2"
    , "H3_description": 'H3', "H4_description": "H4", "H5_description": "H5"}, inplace=True)
sku_details['sku'] = sku_details['sku'].astype(int)

c=sku_details['H1'].isin(categories_present)
sku_details=sku_details[c]

sku_details['NetWeight'] = sku_details['NetWeight'].astype(float)

cond__ = [sku_details['ProductWeightUnit'] == "KG", sku_details['ProductWeightUnit'] == "G",
          sku_details['ProductWeightUnit'] == "MG"]

choices__ = [sku_details['NetWeight'], sku_details['NetWeight'] / 1000, sku_details['NetWeight'] / 1000000]

sku_details['NetWeight_calculated'] = np.select(cond__, choices__)

sku_details['ServiceLevel'] = sku_details['ServiceLevel'].astype(float)

stats_value_map = {
    .50: 0,
    .6: 0.7257,
    .80: 0.842,
    .85: 1.036,
    .90: 1.282,
    .95: 1.645,
    # .99: 2.57,
    # .999: 3.29,
}
sku_details['ServiceLevel_value'] = sku_details['ServiceLevel'].map(stats_value_map)


(246336, 8)
(122511, 4)
(929333, 10)
['part-00000-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet', 'part-00001-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet', 'part-00002-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet', 'part-00003-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet']
./DATA/Product_MASTER/part-00000-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet
(6144, 13)
./DATA/Product_MASTER/part-00001-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet
(6144, 13)
./DATA/Product_MASTER/part-00002-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet
(6691, 13)
./DATA/Product_MASTER/part-00003-5266221d-3be5-4d5e-8122-c2dadcc38471-c000.snappy.parquet
(6144, 13)


In [3]:
c = sku_details['CrossPlantStatus'] == "AC"
active_list = sku_details[c]['sku'].unique()

print(input_data.head())
c = input_data['MATERIAL'].isin(active_list)
input_data = input_data[c]

print(final_forecast_data.head())
c = final_forecast_data['sku'].isin(active_list)
c1 = forecast_history_data['sku'].isin(active_list)
forecast_history_data = forecast_history_data[c1]
print(forecast_history_data.shape, forecast_history_data['sku'].nunique())
final_forecast_data = final_forecast_data[c]

   MATERIAL year_month CHNL_NAME  TOTAL_QTY_BASEUOM_SUM  \
0     101.0    2018-12    Retail                      1   
1     101.0    2019-01       B2B                      9   
2     101.0    2019-01    Retail                      9   
3     101.0    2019-02       B2B                      8   
4     101.0    2019-02    Retail                     10   

   TOTAL_QTY_BASEUOM_COUNT  TOTAL_SALES_SUM  KG_SUM  QTY_BASEUOM_SUM  \
0                        1            14.30     5.0              1.0   
1                        6           121.50    45.0              9.0   
2                        8           125.81    45.0              9.0   
3                        3           108.00    40.0              8.0   
4                        8           138.65    50.0             10.0   

   QTY_BASEUOM_COUNT  SALES_SUM  
0                1.0      14.30  
1                6.0     121.50  
2                8.0     125.81  
3                3.0     108.00  
4                8.0     138.65  
  train_

In [4]:
sku_details.head()
# print(final_forecast_data['year_month'].head())
final_forecast_data['forecast_month'] = final_forecast_data['year_month']
# print(final_forecast_data['forecast_month'].head()+"-01")
print(pd.to_datetime(final_forecast_data['year_month'] + "-01"))
current_date = pd.to_datetime(final_forecast_data['forecast_month'] + "-01").min() - pd.DateOffset(months=1)
print(current_date, "*" * 1000)
input_data['CHNL_NAME'] = np.where(input_data['CHNL_NAME'].isin(['Export', 'Re-Export']), 'Export_Re-Export',
                                   input_data['CHNL_NAME'])

input_data = input_data.groupby(['MATERIAL', 'year_month', 'CHNL_NAME'], as_index=False).sum()

input_data = pd.merge(input_data, sku_details[['sku', "H1"]],
                      left_on=['MATERIAL']
                      , right_on=['sku'], how='left')

c=input_data['H1'].isin(categories_present)
input_data=input_data[c]

input_data.drop(['sku'], axis=1, inplace=True)
print(input_data.head().T)
# c = input_data['H1'].isin(["Dairy"])
# input_data = input_data[c]
print(input_data['H1'].unique(), 'Printing_DATA')
print(final_forecast_data.head(), 'Printing_FF')

0        2024-10-01
1        2024-11-01
2        2024-12-01
3        2025-01-01
4        2025-02-01
            ...    
246217   2024-11-01
246218   2024-12-01
246219   2025-01-01
246220   2025-02-01
246221   2025-03-01
Name: year_month, Length: 130584, dtype: datetime64[ns]
2024-09-01 00:00:00 ********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [5]:
# # TESTING 
# c1=forecast_history_data['sku']==65032
# c2=forecast_history_data['Channel']=="Export_Re-Export"
# display(forecast_history_data[c1 & c2])
# # c=forecast_history_data__1['sku']=="65033"
# # forecast_history_data__1[c]

In [6]:
from utility import make_input_data_continous, collect_data, make_input_data_continous_new

In [7]:
%%time 
input_data_c=make_input_data_continous_new(input_data,current_date)
input_data_c

CPU times: total: 2.95 s
Wall time: 6.59 s


,MATERIAL,CHNL_NAME,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM
0,101.0,Retail,Bakery,2018-12-01,2018-12,1.0,1.0,14.30,5.0,1.0,1.0,14.30
1,101.0,Retail,Bakery,2019-01-01,2019-01,9.0,8.0,125.81,45.0,9.0,8.0,125.81
2,101.0,Retail,Bakery,2019-02-01,2019-02,10.0,8.0,138.65,50.0,10.0,8.0,138.65
3,101.0,Retail,Bakery,2019-03-01,2019-03,10.0,9.0,138.67,50.0,10.0,9.0,138.67
4,101.0,Retail,Bakery,2019-04-01,2019-04,16.0,15.0,220.20,80.0,16.0,15.0,220.20
...,...,...,...,...,...,...,...,...,...,...,...,...
1292315,670176.0,Export_Re-Export,Non Food,2024-11-01,2024-11,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1292316,670176.0,Export_Re-Export,Non Food,2024-12-01,2024-12,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1292317,670176.0,Export_Re-Export,Non Food,2025-01-01,2025-01,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1292318,670176.0,Export_Re-Export,Non Food,2025-02-01,2025-02,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [8]:
input_data_c.sort_values(['MATERIAL', 'H1', "CHNL_NAME", 'date'], inplace=True)
input_data_c.to_csv('./DATA/RESULTS/input_data_.csv', index=False)

In [9]:
# Calculating Last Year Demand for SKU H1 & CHNL_NAME
input_data_c['last_year_demand'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])['TOTAL_QTY_BASEUOM_SUM'].shift(
    12)
input_data_c['last_year_demand_non_contract'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])[
    'QTY_BASEUOM_SUM'].shift(12)
input_data_c['last_year_KG_SUM'] = input_data_c.groupby(['MATERIAL', 'H1', 'CHNL_NAME'])['KG_SUM'].shift(12)

final_forecast_data['year_month'] = final_forecast_data['forecast_month']

In [10]:

input_d = input_data_c.copy()
c = input_d['TOTAL_QTY_BASEUOM_COUNT'] > 0
c1 = input_d['TOTAL_QTY_BASEUOM_SUM'] > 0
input_d = input_d[c & c1]

In [11]:

max_date_input = input_d.groupby(['MATERIAL', 'CHNL_NAME'], as_index=False)['date'].max()
max_date_input.rename(columns={"date": "max_date"}, inplace=True)
max_date_input

,MATERIAL,CHNL_NAME,max_date
0,101.0,B2B,2024-09-01
1,101.0,Ecomm,2024-08-01
2,101.0,Export_Re-Export,2024-08-01
3,101.0,Retail,2024-09-01
4,102.0,B2B,2024-10-01
...,...,...,...
22180,400145.0,Export_Re-Export,2021-06-01
22181,670164.0,B2B,2021-11-01
22182,670174.0,B2B,2021-12-01
22183,670175.0,B2B,2021-12-01


In [12]:
input_d = pd.merge(input_d, max_date_input, on=['MATERIAL', 'CHNL_NAME'], how='left')
input_d

,MATERIAL,CHNL_NAME,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,last_year_demand,last_year_demand_non_contract,last_year_KG_SUM,max_date
0,101.0,B2B,Bakery,2019-01-01,2019-01,9.0,6.0,121.5,45.0,9.0,6.0,121.5,NaN,NaN,NaN,2024-09-01
1,101.0,B2B,Bakery,2019-02-01,2019-02,8.0,3.0,108.0,40.0,8.0,3.0,108.0,NaN,NaN,NaN,2024-09-01
2,101.0,B2B,Bakery,2019-03-01,2019-03,20.0,9.0,264.0,100.0,20.0,9.0,264.0,NaN,NaN,NaN,2024-09-01
3,101.0,B2B,Bakery,2019-04-01,2019-04,5.0,5.0,67.5,25.0,5.0,5.0,67.5,NaN,NaN,NaN,2024-09-01
4,101.0,B2B,Bakery,2019-05-01,2019-05,4.0,4.0,54.0,20.0,4.0,4.0,54.0,NaN,NaN,NaN,2024-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669883,670164.0,B2B,Non Food,2021-10-01,2021-10,8.0,1.0,44.0,4.0,8.0,1.0,44.0,NaN,NaN,NaN,2021-11-01
669884,670164.0,B2B,Non Food,2021-11-01,2021-11,79.0,2.0,434.5,39.5,79.0,2.0,434.5,163.0,163.0,81.5,2021-11-01
669885,670174.0,B2B,Non Food,2021-12-01,2021-12,200.0,1.0,8.0,0.6,200.0,1.0,8.0,NaN,NaN,NaN,2021-12-01
669886,670175.0,B2B,Non Food,2021-12-01,2021-12,200.0,1.0,10.0,1.0,200.0,1.0,10.0,NaN,NaN,NaN,2021-12-01


In [13]:
c = input_d['date'] == input_d['max_date']
input_d = input_d[c][['MATERIAL', 'CHNL_NAME', "TOTAL_SALES_SUM", 'TOTAL_QTY_BASEUOM_SUM']]

In [14]:
input_d["one_unit_cost"] = input_d['TOTAL_SALES_SUM'] / input_d['TOTAL_QTY_BASEUOM_SUM']
input_data_c = pd.merge(input_data_c, input_d[['MATERIAL', 'CHNL_NAME', 'one_unit_cost']], on=['MATERIAL', 'CHNL_NAME'],
                        how='left')

# TILL HERE 

In [15]:

input_data_c.rename(columns={"MATERIAL": "sku", 'CHNL_NAME': "Channel"}, inplace=True)
# input_data_c.to_clipboard(index=False,sep=',')


input_data_c['TOTAL_QTY_BASEUOM_SUM'].fillna(0, inplace=True)
input_data_c['QTY_BASEUOM_SUM'].fillna(0, inplace=True)
input_data_c['contract_sales'] = input_data_c['TOTAL_QTY_BASEUOM_SUM'] - input_data_c['QTY_BASEUOM_SUM']


In [16]:

input_data_c.rename(columns={"MATERIAL": "sku", 'CHNL_NAME': "Channel"}, inplace=True)
# input_data_c.to_clipboard(index=False,sep=',')


input_data_c['TOTAL_QTY_BASEUOM_SUM'].fillna(0, inplace=True)
input_data_c['QTY_BASEUOM_SUM'].fillna(0, inplace=True)
input_data_c['contract_sales'] = input_data_c['TOTAL_QTY_BASEUOM_SUM'] - input_data_c['QTY_BASEUOM_SUM']

# SCHEMA PREDICTION VS LAST YEAR DEMAND - START
final_file_schema_1 = pd.merge(final_forecast_data,
                               input_data_c[
                                   ['sku', 'year_month', 'H1', 'Channel', 'last_year_demand_non_contract',
                                    'last_year_demand', 'one_unit_cost']],
                               on=['sku', 'H1', 'Channel', 'year_month'], how='left')

final_file_schema_1 = pd.merge(final_file_schema_1, sku_details[['sku', 'NetWeight']], on='sku', how='left')
final_file_schema_1['NetWeight'] = final_file_schema_1['NetWeight'].astype(float)

data_mapping_prediction_vs_last_year_demand = get_datamapping_pred_vs_last_year_demand(final_file_schema_1)


In [17]:
final_file_schema_1['Revised_Sale'] = 0

# SCHEMA 1 PREDICTION VS LAST YEAR DEMAND
pred_vs_lyd_schema = generate_prediction_vs_last_year_demand(final_file_schema_1,
                                                             data_mapping_prediction_vs_last_year_demand)

pred_vs_lyd_schema.to_csv('./DATA/RESULTS/pred_vslast.csv', index=False, sep=',')

pred_vs_lyd_schema['prediction'] = np.ceil(pred_vs_lyd_schema['prediction'])

pred_vs_lyd_schema['last_year_demand'] = np.ceil(pred_vs_lyd_schema['last_year_demand'])

pred_vs_lyd_schema.to_excel(writer, sheet_name=sheet_name + "PRED_VS_LAST_YEAR_DEMAND", index=False)


In [18]:
c = input_data_c['date'] <= current_date
c2 = input_data_c['date'] > current_date - pd.DateOffset(months=6)
summary_product_sales_data = input_data_c[c & c2]
summary_product_sales_data

,sku,Channel,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,last_year_demand,last_year_demand_non_contract,last_year_KG_SUM,one_unit_cost,contract_sales
63,101.0,B2B,Bakery,2024-04-01,2024-04,14.0,6.0,189.0,70.0,14.0,6.0,189.0,5.0,5.0,25.0,13.50,0.0
64,101.0,B2B,Bakery,2024-05-01,2024-05,12.0,6.0,162.0,60.0,12.0,6.0,162.0,7.0,7.0,35.0,13.50,0.0
65,101.0,B2B,Bakery,2024-06-01,2024-06,9.0,7.0,121.5,45.0,9.0,7.0,121.5,5.0,5.0,25.0,13.50,0.0
66,101.0,B2B,Bakery,2024-07-01,2024-07,8.0,4.0,108.0,40.0,8.0,4.0,108.0,22.0,22.0,110.0,13.50,0.0
67,101.0,B2B,Bakery,2024-08-01,2024-08,12.0,5.0,162.0,60.0,12.0,5.0,162.0,22.0,22.0,110.0,13.50,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292309,670176.0,Export_Re-Export,Non Food,2024-05-01,2024-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.0
1292310,670176.0,Export_Re-Export,Non Food,2024-06-01,2024-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.0
1292311,670176.0,Export_Re-Export,Non Food,2024-07-01,2024-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.0
1292312,670176.0,Export_Re-Export,Non Food,2024-08-01,2024-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.12,0.0


In [19]:

summary_product_sales_data = pd.merge(summary_product_sales_data, sku_details[['sku', 'NetWeight']], on=['sku'],
                                      how='left')
print(summary_product_sales_data.head().T)
summary_product_sales_data['NetWeight'] = summary_product_sales_data['NetWeight'].astype(float)

data_mapping_summary_product_sales = get_data_mapping_summary_product_sales(input_data_c)

# SCHEMA 2 SUMMARY PRODUCT SALES
product_sales_schema_data = generate_summary_product_sales(summary_product_sales_data,
                                                           data_mapping_summary_product_sales)
product_sales_schema_data['client_id'] = 1

product_sales_schema_data['last_updated_at'] = datetime.now().date()

# product_sales_schema_data["Current Sales"] = np.ceil(product_sales_schema_data["Current Sales"])
product_sales_schema_data["Current Sales"] = product_sales_schema_data["Current Sales"].astype(int)
# product_sales_schema_data["Last Year Sales"] = np.ceil(product_sales_schema_data["Last Year Sales"])
product_sales_schema_data["Last Year Sales"] = product_sales_schema_data["Last Year Sales"].astype(int)

product_sales_schema_data.rename(columns={
    "Current Sales": "current_sales",
    "Last Year Sales": "last_year_sales",
    "H1": "h1",
    "Channel": "channel",
    "Units_Base_UOM": "units_base_uom",
    "Value": "value",
}, inplace=True)

product_sales_schema_data['year_month'] = product_sales_schema_data['year_month'] + "-01"

product_sales_schema_data.to_excel(writer, sheet_name=sheet_name + "PRODUCT_SALES", index=False)


                                                 0                    1  \
sku                                          101.0                101.0   
Channel                                        B2B                  B2B   
H1                                          Bakery               Bakery   
date                           2024-04-01 00:00:00  2024-05-01 00:00:00   
year_month                                 2024-04              2024-05   
TOTAL_QTY_BASEUOM_SUM                         14.0                 12.0   
TOTAL_QTY_BASEUOM_COUNT                        6.0                  6.0   
TOTAL_SALES_SUM                              189.0                162.0   
KG_SUM                                        70.0                 60.0   
QTY_BASEUOM_SUM                               14.0                 12.0   
QTY_BASEUOM_COUNT                              6.0                  6.0   
SALES_SUM                                    189.0                162.0   
last_year_demand         

In [20]:
product_sales_schema_data.to_csv('./DATA/RESULTS/prod_summary.csv', index=False, sep=',')

In [21]:
forecast_history_data

,sku,year_month,final_perdiction,Channel
0,1039,2024-07,1.080000e-78,Export_Re-Export
1,1087,2024-07,1.012576e+02,Export_Re-Export
2,1088,2024-07,2.622755e+02,Export_Re-Export
3,1095,2024-07,4.944778e+02,Export_Re-Export
4,1917,2024-07,2.944100e+02,Export_Re-Export
...,...,...,...,...
122478,81186,2024-09,7.980000e-26,Ecomm
122479,81187,2024-09,7.980000e-25,Ecomm
122486,100350,2024-09,0.000000e+00,Ecomm
122490,104955,2024-09,0.000000e+00,Ecomm


In [22]:
sku_channel_history = forecast_history_data[['sku', 'Channel']].drop_duplicates()
sku_channel_history.shape

(21703, 2)

In [23]:
history_date_range=(pd.date_range(current_date - pd.DateOffset(months=6), current_date, freq="M") + pd.DateOffset(days=1)).astype(str).str[:-3]
print(history_date_range)

sku_channel_history['grp']=1
sku_channel_history.head()

dummy_data=pd.DataFrame({'grp':[1],'year_month':[history_date_range]})
sku_channel_history_data_=pd.merge(sku_channel_history,dummy_data,on=['grp'],how='left')
sku_channel_history_data_=sku_channel_history_data_.explode('year_month')
sku_channel_history_data_.shape

Index(['2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09'], dtype='object')


(130218, 4)

In [24]:
# forecast_history_data

In [25]:
forecast_history_data.dtypes

sku                   int64
year_month           object
final_perdiction    float64
Channel              object
dtype: object

In [26]:
# TSTING STARSt

sku_channel_history_data_.reset_index(drop=True,inplace=True)
# TSTING STOP
del sku_channel_history_data_['grp']
sku_channel_history_data_

,sku,Channel,year_month
0,1039,Export_Re-Export,2024-04
1,1039,Export_Re-Export,2024-05
2,1039,Export_Re-Export,2024-06
3,1039,Export_Re-Export,2024-07
4,1039,Export_Re-Export,2024-08
...,...,...,...
130213,111259,Export_Re-Export,2024-05
130214,111259,Export_Re-Export,2024-06
130215,111259,Export_Re-Export,2024-07
130216,111259,Export_Re-Export,2024-08


In [27]:
sku_channel_history_data_.isna().mean()

sku           0.0
Channel       0.0
year_month    0.0
dtype: float64

In [28]:
forecast_history_data.isna().mean()

sku                 0.0
year_month          0.0
final_perdiction    0.0
Channel             0.0
dtype: float64

In [29]:
# sku_channel_history_data_['sku']=sku_channel_history_data_['sku'].astype(str)
# forecast_history_data['sku']=forecast_history_data['sku'].astype(str)

In [30]:
sku_channel_history_data_.dtypes

sku            int64
Channel       object
year_month    object
dtype: object

In [31]:
forecast_history_data['final_perdiction']=forecast_history_data['final_perdiction'].astype(int)

In [32]:
# # TESTING START
# forecast_history_data__1 = pd.merge(sku_channel_history_data_, forecast_history_data, on=['sku',"Channel" ,'year_month'],
#                                  how='left')
# forecast_history_data__1.shape
# forecast_history_data__1.to_clipboard(index=False)
# # TESTING START

In [33]:
# # TESTING 
# c1=forecast_history_data['sku']=="65032"
# c2=forecast_history_data['Channel']=="Export_Re-Export"
# display(forecast_history_data[c1 & c2])
# c=forecast_history_data__1['sku']=="65033"
# forecast_history_data__1[c]

In [34]:
forecast_history_data = pd.merge(sku_channel_history_data_, forecast_history_data, on=['sku', 'year_month', 'Channel'],
                                 how='left')
forecast_history_data.shape

(130218, 4)

# TILL HERE

In [35]:

forecast_history_data_ = forecast_history_data[forecast_history_data['final_perdiction'].notnull()]

In [36]:
accuracy_segmentation = pd.merge(forecast_history_data_,
                                 input_data_c[['sku', 'year_month', 'Channel', 'TOTAL_QTY_BASEUOM_SUM',
                                               'TOTAL_SALES_SUM', 'contract_sales']],
                                 on=['sku', 'year_month', 'Channel'], how='left')

accuracy_segmentation['TOTAL_QTY_BASEUOM_SUM'].fillna(0, inplace=True)
accuracy_segmentation['TOTAL_SALES_SUM'].fillna(0, inplace=True)



In [37]:
accuracy_segmentation['Revision_Sales'] = 0

acc_final_data = pd.merge(accuracy_segmentation, sku_details[['sku', 'H1', 'NetWeight_calculated']], on='sku',
                          how='left')
print(acc_final_data)

acc_final_data.rename(columns={"TOTAL_QTY_BASEUOM_SUM": "actual_demand"}, inplace=True)

acc_final_data['final_perdiction'] = acc_final_data['final_perdiction'] + acc_final_data["contract_sales"]

acc_final_data['mape'] = np.abs(acc_final_data['actual_demand'] - acc_final_data['final_perdiction']) / \
                         acc_final_data[
                             'actual_demand']
c1 = acc_final_data['mape'] <= .25
c2 = acc_final_data['mape'] > 0.5
c3 = (acc_final_data['mape'] > 0.25) & (acc_final_data['mape'] <= 0.5)
acc_final_data["ACC_BUCKET"] = np.select([c1, c2, c3], ["HIGH", "LOW", 'MEDIUM'], "NA")
acc_final_data["ACC_BUCKET"] = np.where(np.isinf(acc_final_data['mape']), 'NA', acc_final_data['ACC_BUCKET'])
print(acc_final_data['mape'].isna().sum())
acc_final_data["mape"] = np.where(np.isinf(acc_final_data['mape']), 'NA', acc_final_data['mape'])
acc_final_data["mape"] = np.where(acc_final_data['mape'].isna(), 'NA', acc_final_data['mape'])

data_mapping_accuracy_segmentation = get_data_mapping_accuracy_segmentation(input_data_c)
# SCHEMA 3 ACCURACY SEGMENTATION
acc_final_data_schema = get_accuracy_segmentation_schema(acc_final_data, data_mapping_accuracy_segmentation)
# CORRECTION ACC SEG COUNT << START
c = sku_details['CrossPlantStatus'] == "AC"
add_all_data = {
    "H1": ["ALL"],
    'sku_count': [c.sum()]
}
h1_count = sku_details[c].groupby('H1', as_index=False)["sku"].count()
h1_count.rename(columns={'sku': 'sku_count'}, inplace=True)
h1_count = pd.concat([pd.DataFrame(add_all_data), h1_count], axis=0)
h1_count
acc_final_data_schema = pd.merge(acc_final_data_schema, h1_count, on=['H1'], how='left')
# acc_final_data_schema['count_sku_NA'] = acc_final_data_schema["sku_count"] - (
#             acc_final_data_schema['count_sku_HIGH'] + acc_final_data_schema['count_sku_LOW'] + acc_final_data_schema[
#         'count_sku_MEDIUM'])
# CORRECTION ACC SEG COUNT << END

acc_final_data_schema["client_id"] = 1
acc_final_data_schema['last_updated_at'] = datetime.now().date()

rename_col_for_acc_seg = {
    "sum_SALES_TOTAL_HIGH": "high_percentage_value",
    "sum_SALES_TOTAL_MEDIUM": "medium_percentage_value",
    "sum_SALES_TOTAL_LOW": "low_percentage_value",
    "sum_SALES_TOTAL_NA": "others_percentage_value",
    "count_sku_HIGH": "high_sku_count",
    "count_sku_MEDIUM": "medium_sku_count",
    "count_sku_LOW": "low_sku_count",
    "count_sku_NA": "others_sku_count",
    "H1": "h1",
    "Channel": "channel",
    "Statistical Accuracy": "statistical_accuracy",
    "Total Accuracy": "total_accuracy",
}

acc_final_data_schema.rename(columns=rename_col_for_acc_seg, inplace=True)

columns = ['year_month', 'high_percentage_value', 'low_percentage_value',
           'medium_percentage_value', 'others_percentage_value', 'high_sku_count',
           'low_sku_count', 'medium_sku_count', 'others_sku_count', 'channel',
           'h1', 'statistical_accuracy', 'total_accuracy', 'client_id',
           'last_updated_at']
actual_columns = set(columns)

columns_present = set(acc_final_data_schema.columns)

for i in actual_columns - columns_present:
    acc_final_data_schema[i] = np.nan

# acc_final_data_schema['year_month'] = pd.to_datetime(acc_final_data_schema['year_month'] + "-01")
acc_final_data_schema['year_month'] = acc_final_data_schema['year_month'] + "-01"

acc_final_data_schema["high_percentage_value"] = np.round(acc_final_data_schema['high_percentage_value'], decimals=2)
acc_final_data_schema["medium_percentage_value"] = np.round(acc_final_data_schema['medium_percentage_value'],
                                                            decimals=2)
acc_final_data_schema["low_percentage_value"] = np.round(acc_final_data_schema['low_percentage_value'], decimals=2)

print("Acc final Data Schema")
acc_final_data_schema.to_csv("./DATA/RESULTS/acc_seg.csv")
acc_final_data_schema.to_excel(writer, sheet_name=sheet_name + "ACC_SEG", index=False)



          sku           Channel year_month  final_perdiction  \
0        1039  Export_Re-Export    2024-07               0.0   
1        1039  Export_Re-Export    2024-08               0.0   
2        1039  Export_Re-Export    2024-09               0.0   
3        1087  Export_Re-Export    2024-07             101.0   
4        1087  Export_Re-Export    2024-08             100.0   
...       ...               ...        ...               ...   
64814  110806               B2B    2024-09              10.0   
64815  110806            Retail    2024-09              29.0   
64816   80384  Export_Re-Export    2024-09               0.0   
64817  110396               B2B    2024-09            2354.0   
64818  111259  Export_Re-Export    2024-09               3.0   

       TOTAL_QTY_BASEUOM_SUM  TOTAL_SALES_SUM  contract_sales  Revision_Sales  \
0                        0.0             0.00             0.0               0   
1                        0.0             0.00             0.0        

In [38]:
acc_final_data.to_csv("./DATA/acc_final.csv")
forecast_accuracy_schema = get_forecast_accuracy_schema(acc_final_data, data_mapping_accuracy_segmentation)
forecast_accuracy_schema.rename(columns={"Predicted": "predicted"}, inplace=True)

forecast_accuracy_schema['year_month'] = forecast_accuracy_schema['year_month'] + '-01'
forecast_accuracy_schema['mape'] = 1 - forecast_accuracy_schema['mape']
forecast_accuracy_schema['mape'] = np.where(forecast_accuracy_schema['mape'] < 0,
                                            0, forecast_accuracy_schema['mape'])

forecast_accuracy_schema['client_id'] = 1
forecast_accuracy_schema['last_updated_at'] = datetime.now().date()
print("Acc forecast accuracy schema")
forecast_accuracy_schema.to_csv("./DATA/RESULTS/fore_acc.csv")
forecast_accuracy_schema.to_excel(writer, sheet_name=sheet_name + "FORECAST_ACC", index=False)
# SCHEMA ACCURACY SEGMENTATION & Summary: Forecast Accuracy END


acc_final_data['TOTAL_SALES_SUM'].fillna(0, inplace=True)
acc_final_data['actual_demand'].fillna(0, inplace=True)

data_mapping_confusion_matrix = get_data_mapping_confusion_matrix(input_data_c)

c = data_mapping_confusion_matrix['Channel'] != "ALL"
data_mapping_confusion_matrix = data_mapping_confusion_matrix[c]

# SCHEMA 5 CONFUSION MATRIX
acc_final_data = pd.merge(acc_final_data,
                          input_data_c[['sku', 'Channel', 'one_unit_cost']].drop_duplicates(),
                          on=['sku', 'Channel'],
                          how='left')
acc_final_data["actual_sales"] = acc_final_data['actual_demand'] * acc_final_data['one_unit_cost']
acc_final_data.to_csv('acc_final.csv', sep=',')
confusion_matrix_schema = get_confusion_matrix_schema(acc_final_data, data_mapping_confusion_matrix)
# confusion_matrix_schema = get_confusion_matrix_schema_new(acc_final_data, data_mapping_confusion_matrix, current_date)
confusion_matrix_schema['client_id'] = 1
confusion_matrix_schema['last_updated_at'] = datetime.now().date()
confusion_matrix_schema['year_month'] = confusion_matrix_schema['year_month'] + "-01"
confusion_matrix_schema.rename(columns={
    "Actual Sales": "actual_sales",
    "Predicted Sales": "predicted_sales",
    "No of SKUS": "no_of_skus",
    "H1": "h1",
    "Channel": "channel",
    "Total": "total",
    "Statistical": "statistical",

}, inplace=True)

print("Acc confusion matrix schema")
confusion_matrix_schema.to_csv('./DATA/RESULTS/confusion_matrix.csv')
confusion_matrix_schema.to_excel(writer, sheet_name=sheet_name + "CONFUSION_MATRIX", index=False)



Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total Accuracy
Statistical Accuracy
Total A

# TILL HERE 

In [39]:
forecast_to_attach=pd.concat([forecast_history_data_,final_forecast_data[['sku','year_month','Channel','final_perdiction']]])
input_data_c1=pd.merge(input_data_c,forecast_to_attach,on=['sku','year_month','Channel'],how='left')
input_data_c1['mape'] = np.abs(input_data_c1['final_perdiction'] - input_data_c1['QTY_BASEUOM_SUM']) / input_data_c1['QTY_BASEUOM_SUM']
input_data_c1['notinf'] = np.where(np.isinf(input_data_c1['mape']), 0, 1)
input_data_c1['mape'] = np.where(np.isinf(input_data_c1['mape']), 0, input_data_c1['mape'])

input_data_c1.sort_values(['sku','Channel','year_month'],inplace=True)
input_data_c1['3_mape']=input_data_c1.groupby(['sku','Channel'])['mape'].rolling(3).sum().values
input_data_c1['3_notinf']=input_data_c1.groupby(['sku','Channel'])['notinf'].rolling(3).sum().values
input_data_c1['3_months_acc'] = input_data_c1['3_mape'] / input_data_c1['3_notinf']
input_data_c1['3_months_acc']=np.where(input_data_c1['date']>current_date,np.nan,input_data_c1['3_months_acc'])
input_data_c1['TOTAL_QTY_BASEUOM_SUM']=np.where(input_data_c1['date']>current_date,np.nan,input_data_c1['TOTAL_QTY_BASEUOM_SUM'])

input_data_c1

,sku,Channel,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,...,last_year_demand_non_contract,last_year_KG_SUM,one_unit_cost,contract_sales,final_perdiction,mape,notinf,3_mape,3_notinf,3_months_acc
0,101.0,B2B,Bakery,2019-01-01,2019-01,9.0,6.0,121.5,45.0,9.0,...,NaN,NaN,13.50,0.0,NaN,NaN,1,NaN,NaN,NaN
1,101.0,B2B,Bakery,2019-02-01,2019-02,8.0,3.0,108.0,40.0,8.0,...,NaN,NaN,13.50,0.0,NaN,NaN,1,NaN,NaN,NaN
2,101.0,B2B,Bakery,2019-03-01,2019-03,20.0,9.0,264.0,100.0,20.0,...,NaN,NaN,13.50,0.0,NaN,NaN,1,NaN,3.0,NaN
3,101.0,B2B,Bakery,2019-04-01,2019-04,5.0,5.0,67.5,25.0,5.0,...,NaN,NaN,13.50,0.0,NaN,NaN,1,NaN,3.0,NaN
4,101.0,B2B,Bakery,2019-05-01,2019-05,4.0,4.0,54.0,20.0,4.0,...,NaN,NaN,13.50,0.0,NaN,NaN,1,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292315,670176.0,Export_Re-Export,Non Food,2024-11-01,2024-11,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12,0.0,0.000043,0.0,0,NaN,1.0,NaN
1292316,670176.0,Export_Re-Export,Non Food,2024-12-01,2024-12,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12,0.0,0.000043,0.0,0,0.0,0.0,NaN
1292317,670176.0,Export_Re-Export,Non Food,2025-01-01,2025-01,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12,0.0,0.000043,0.0,0,0.0,0.0,NaN
1292318,670176.0,Export_Re-Export,Non Food,2025-02-01,2025-02,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.12,0.0,0.000043,0.0,0,0.0,0.0,NaN


In [40]:
# Attach _revision here 
input_data_c1['revision']=0

In [41]:
input_data_c1['statistical_forecast']=input_data_c1['final_perdiction']
input_data_c1.head().T

,0,1,2,3,4
sku,101.0,101.0,101.0,101.0,101.0
Channel,B2B,B2B,B2B,B2B,B2B
H1,Bakery,Bakery,Bakery,Bakery,Bakery
date,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00
year_month,2019-01,2019-02,2019-03,2019-04,2019-05
TOTAL_QTY_BASEUOM_SUM,9.0,8.0,20.0,5.0,4.0
TOTAL_QTY_BASEUOM_COUNT,6.0,3.0,9.0,5.0,4.0
TOTAL_SALES_SUM,121.5,108.0,264.0,67.5,54.0
KG_SUM,45.0,40.0,100.0,25.0,20.0
QTY_BASEUOM_SUM,9.0,8.0,20.0,5.0,4.0


In [42]:
input_data_c1['final_forecast'] = input_data_c1['statistical_forecast'].fillna(0).astype(float).astype(int) + input_data_c1['revision'].astype(int)

In [43]:
# print(sku_demand_schema['statistical'])
# print(sku_demand_schema['statistical'].unique())
input_data_c1['statistical'] = '1'
input_data_c1['last_updated_at'] = datetime.now().date()


In [44]:
input_data_c1=pd.merge(input_data_c1,sku_details[['sku','ServiceLevel','ServiceLevel_value',"LeadTime"]],on=['sku'],how='left')

In [45]:
c=input_data_c1['date']<=current_date
c1=input_data_c1['date']>(current_date-pd.DateOffset(months=24))
sku_wise_data=input_data_c[c & c1][['sku','year_month','TOTAL_QTY_BASEUOM_SUM']].groupby(['sku','year_month'],as_index=False)['TOTAL_QTY_BASEUOM_SUM'].sum()
std_dev=sku_wise_data.groupby(['sku'],as_index=False).agg({'TOTAL_QTY_BASEUOM_SUM':['std','var','mean']})
std_dev.columns=['sku','Standard_Dev','var','mean']
std_dev['Variability']=std_dev['Standard_Dev']/std_dev['mean']
input_data_c1=pd.merge(input_data_c1,std_dev,on=['sku'],how='left')
input_data_c1

,sku,Channel,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,...,final_forecast,statistical,last_updated_at,ServiceLevel,ServiceLevel_value,LeadTime,Standard_Dev,var,mean,Variability
0,101.0,B2B,Bakery,2019-01-01,2019-01,9.0,6.0,121.5,45.0,9.0,...,0,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776
1,101.0,B2B,Bakery,2019-02-01,2019-02,8.0,3.0,108.0,40.0,8.0,...,0,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776
2,101.0,B2B,Bakery,2019-03-01,2019-03,20.0,9.0,264.0,100.0,20.0,...,0,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776
3,101.0,B2B,Bakery,2019-04-01,2019-04,5.0,5.0,67.5,25.0,5.0,...,0,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776
4,101.0,B2B,Bakery,2019-05-01,2019-05,4.0,4.0,54.0,20.0,4.0,...,0,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292315,670176.0,Export_Re-Export,Non Food,2024-11-01,2024-11,NaN,0.0,0.0,0.0,0.0,...,0,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899
1292316,670176.0,Export_Re-Export,Non Food,2024-12-01,2024-12,NaN,0.0,0.0,0.0,0.0,...,0,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899
1292317,670176.0,Export_Re-Export,Non Food,2025-01-01,2025-01,NaN,0.0,0.0,0.0,0.0,...,0,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899
1292318,670176.0,Export_Re-Export,Non Food,2025-02-01,2025-02,NaN,0.0,0.0,0.0,0.0,...,0,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899


In [46]:
input_data_c1['Safety Stock'] = np.sqrt(np.ceil(input_data_c1['LeadTime'].astype(float) / 30)) * input_data_c1['ServiceLevel_value'].astype(float) * (input_data_c1['Standard_Dev']) * 1.2

In [47]:
# input_data_c1.head().T

In [48]:
c=input_data_c1['date']> (current_date-pd.DateOffset(months=2))
sku_demand_schema=input_data_c1[c]
sku_demand_schema

,sku,Channel,H1,date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,...,statistical,last_updated_at,ServiceLevel,ServiceLevel_value,LeadTime,Standard_Dev,var,mean,Variability,Safety Stock
67,101.0,B2B,Bakery,2024-08-01,2024-08,12.0,5.0,162.0,60.0,12.0,...,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776,8.720680
68,101.0,B2B,Bakery,2024-09-01,2024-09,12.0,6.0,162.0,60.0,12.0,...,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776,8.720680
69,101.0,B2B,Bakery,2024-10-01,2024-10,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776,8.720680
70,101.0,B2B,Bakery,2024-11-01,2024-11,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776,8.720680
71,101.0,B2B,Bakery,2024-12-01,2024-12,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.80,0.842,4.0,8.630918,74.492754,22.666667,0.380776,8.720680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292315,670176.0,Export_Re-Export,Non Food,2024-11-01,2024-11,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899,705.949256
1292316,670176.0,Export_Re-Export,Non Food,2024-12-01,2024-12,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899,705.949256
1292317,670176.0,Export_Re-Export,Non Food,2025-01-01,2025-01,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899,705.949256
1292318,670176.0,Export_Re-Export,Non Food,2025-02-01,2025-02,NaN,0.0,0.0,0.0,0.0,...,1,2024-10-21,0.95,1.645,75.0,206.474160,42631.578947,47.368421,4.358899,705.949256


In [49]:
sku_demand_schema['total']=0

In [50]:
# sku_demand_schema['statistical_forecast']

In [51]:
# sku_details

In [52]:
sku_demand_schema=pd.merge(sku_demand_schema,sku_details[['sku','H2','MrpPurchaser','PdProductDescription']],on=['sku'],how='left')

In [53]:
# REquired
sku_demand_schema['year'] = pd.to_datetime(sku_demand_schema['year_month'] + "-01").dt.year
sku_demand_schema['month'] = pd.to_datetime(sku_demand_schema['year_month'] + "-01").dt.month

rename_dict = {
    # "year_month":"",
    "PdProductDescription": "product_name",
    "H1": "h1",
    "H2": "h2",
    "Channel": "channel",
    "MrpPurchaser": "mrp_purchaser",
    "ServiceLevel": "desired_service_level",
    "Variability": "variability",
    "3_months_acc": "three_month_accuracy",
    "Safety Stock": "safety_stock",
    "Revision_Sales": "revision",
    # "final_perdiction": "statistical_forecast",
    "Predicted": "final_forecast",
    "TOTAL_QTY_BASEUOM_SUM": "actuals",
    "Statistical": "statistical",
    "Total": "total",
}
sku_demand_schema.rename(columns=rename_dict, inplace=True)


In [54]:

sku_demand_schema['client_id'] = 1
cols_req = ['sku', 'product_name', 'h1', 'h2', 'channel', 'mrp_purchaser',
            'desired_service_level', 'variability', 'three_month_accuracy', 'revision', 'contract_sales',
            'final_forecast', 'mape',
            'month', 'year', 'statistical_forecast', 'last_year_demand', 'actuals',
            'safety_stock', 'statistical', 'total', 'client_id']
sku_demand_schema=sku_demand_schema[cols_req]
sku_demand_schema

,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,mape,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id
0,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.166667,8,2024,10.000000,22.0,12.0,8.720680,1,0,1
1,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,0.138889,0,...,0.000000,9,2024,12.000000,11.0,12.0,8.720680,1,0,1
2,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,10,2024,13.000000,15.0,NaN,8.720680,1,0,1
3,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,11,2024,13.000000,15.0,NaN,8.720680,1,0,1
4,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,12,2024,13.000000,13.0,NaN,8.720680,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177676,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,11,2024,0.000043,0.0,NaN,705.949256,1,0,1
177677,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,12,2024,0.000043,0.0,NaN,705.949256,1,0,1
177678,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,1,2025,0.000043,0.0,NaN,705.949256,1,0,1
177679,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,2,2025,0.000043,0.0,NaN,705.949256,1,0,1


In [55]:
sku_demand_schema['statistical_forecast'] = np.round(sku_demand_schema['statistical_forecast'])
# .astype(int, errors='ignore'))
sku_demand_schema['safety_stock'] = np.ceil(sku_demand_schema['safety_stock'])
sku_demand_schema['mape'] = np.where(np.isinf(sku_demand_schema['mape']), np.nan, sku_demand_schema['mape'])
sku_demand_schema

,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,mape,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id
0,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.166667,8,2024,10.0,22.0,12.0,9.0,1,0,1
1,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,0.138889,0,...,0.000000,9,2024,12.0,11.0,12.0,9.0,1,0,1
2,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,10,2024,13.0,15.0,NaN,9.0,1,0,1
3,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,11,2024,13.0,15.0,NaN,9.0,1,0,1
4,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,0.000000,12,2024,13.0,13.0,NaN,9.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177676,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,11,2024,0.0,0.0,NaN,706.0,1,0,1
177677,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,12,2024,0.0,0.0,NaN,706.0,1,0,1
177678,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,1,2025,0.0,0.0,NaN,706.0,1,0,1
177679,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,0.000000,2,2025,0.0,0.0,NaN,706.0,1,0,1


# RUN TILL HERE

In [56]:
sku_demand_schema.dtypes

sku                      float64
product_name              object
h1                        object
h2                        object
channel                   object
mrp_purchaser             object
desired_service_level    float64
variability              float64
three_month_accuracy     float64
revision                   int64
contract_sales           float64
final_forecast             int32
mape                     float64
month                      int32
year                       int32
statistical_forecast     float64
last_year_demand         float64
actuals                  float64
safety_stock             float64
statistical               object
total                      int64
client_id                  int64
dtype: object

In [57]:
sku_demand_schema['final_forecast'] = sku_demand_schema['statistical_forecast'].fillna(0).astype(float).astype(int) + sku_demand_schema[
    'revision'].astype(int)
# sku_demand_schema

## TILL HERE 

In [58]:
statistical_c = sku_demand_schema['statistical'] == '1'
sku_demand_schema['last_updated_at'] = datetime.now().date()

sku_demand_schema

,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at
0,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,8,2024,10.0,22.0,12.0,9.0,1,0,1,2024-10-21
1,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,0.138889,0,...,9,2024,12.0,11.0,12.0,9.0,1,0,1,2024-10-21
2,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,10,2024,13.0,15.0,NaN,9.0,1,0,1,2024-10-21
3,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,11,2024,13.0,15.0,NaN,9.0,1,0,1,2024-10-21
4,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.80,0.380776,NaN,0,...,12,2024,13.0,13.0,NaN,9.0,1,0,1,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177676,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,11,2024,0.0,0.0,NaN,706.0,1,0,1,2024-10-21
177677,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,12,2024,0.0,0.0,NaN,706.0,1,0,1,2024-10-21
177678,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,1,2025,0.0,0.0,NaN,706.0,1,0,1,2024-10-21
177679,670176.0,LIDS TAMPER EVIDENT TE118/LID (RED),Non Food,Raw Material,Export_Re-Export,SENG LIAN,0.95,4.358899,NaN,0,...,2,2025,0.0,0.0,NaN,706.0,1,0,1,2024-10-21


In [62]:
sku_demand_page = sku_demand_schema.copy()

In [64]:
# sku_demand_page

In [66]:
# Adding Extra zero sales sku >>

sample_sku_detail = sku_demand_page.iloc[0, :]
sku_ = sample_sku_detail['sku']
channel_ = sample_sku_detail['channel']
channel_ = sample_sku_detail['channel']
c = sku_demand_page['sku'] == sku_
c1 = sku_demand_page['channel'] == channel_
print(sku_, channel_)
sample_sku_schema = sku_demand_page[c & c1].drop_duplicates()

sample_sku_schema['three_month_accuracy'] = np.nan
sample_sku_schema['statistical'] = 0
sample_sku_schema['safety_stock'] = np.nan
sample_sku_schema['last_year_demand'] = 0
sample_sku_schema['contract_sales'] = 0
sample_sku_schema['revision'] = 0
sample_sku_schema['actuals'] = 0
sample_sku_schema['total'] = 0
sample_sku_schema['final_forecast'] = 0
sample_sku_schema['mape'] = np.nan
sample_sku_schema['variability'] = 0
sample_sku_schema['statistical_forecast'] = 0

a = set(sku_demand_page['sku'].unique())
b = set(sku_details[sku_details['CrossPlantStatus'] == "AC"]['sku'].unique())
a,b


from tqdm import tqdm
extra_sku = pd.DataFrame()
for i in tqdm(b - a):
    c = sku_details['sku'] == i
    sku_details_ = sku_details[c]
    dummy_2 = sample_sku_schema.copy()
    # for j in sku_demand['channel'].unique():
    for j in ["B2B", "Ecomm", "Retail", "Export_Re-Export"]:
        dummy_2['sku'] = i
        dummy_2['channel'] = j
        dummy_2['product_name'] = sku_details_['PdProductDescription'].values[0]
        dummy_2['desired_service_level'] = sku_details_['ServiceLevel'].values[0]
        dummy_2['h1'] = sku_details_['H1'].values[0]
        dummy_2['h2'] = sku_details_['H2'].values[0]
    extra_sku = pd.concat([extra_sku, dummy_2], axis=0)

extra_sku

101.0 B2B


100%|█████████████████████████████████████████████████████████████████████████████| 1742/1742 [00:08<00:00, 216.23it/s]


,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at
0,65556,QUASER 20KG,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,8,2024,0,0,0,NaN,0,0,1,2024-10-21
1,65556,QUASER 20KG,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,9,2024,0,0,0,NaN,0,0,1,2024-10-21
2,65556,QUASER 20KG,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,10,2024,0,0,0,NaN,0,0,1,2024-10-21
3,65556,QUASER 20KG,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,11,2024,0,0,0,NaN,0,0,1,2024-10-21
4,65556,QUASER 20KG,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,12,2024,0,0,0,NaN,0,0,1,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,57334,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,11,2024,0,0,0,NaN,0,0,1,2024-10-21
4,57334,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,12,2024,0,0,0,NaN,0,0,1,2024-10-21
5,57334,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,1,2025,0,0,0,NaN,0,0,1,2024-10-21
6,57334,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0,NaN,0,...,2,2025,0,0,0,NaN,0,0,1,2024-10-21


In [76]:
sku_demand=pd.concat([sku_demand_page,extra_sku.reset_index(drop=True)],axis=0)
sku_demand

,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at
0,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.8,0.380776,NaN,0,...,8,2024,10.0,22.0,12.0,9.0,1,0,1,2024-10-21
1,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.8,0.380776,0.138889,0,...,9,2024,12.0,11.0,12.0,9.0,1,0,1,2024-10-21
2,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.8,0.380776,NaN,0,...,10,2024,13.0,15.0,NaN,9.0,1,0,1,2024-10-21
3,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.8,0.380776,NaN,0,...,11,2024,13.0,15.0,NaN,9.0,1,0,1,2024-10-21
4,101.0,AMMONIUM BICARBONATE REDMAN 5KG,Bakery,Functional,B2B,LESTER SEOW,0.8,0.380776,NaN,0,...,12,2024,13.0,13.0,NaN,9.0,1,0,1,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13931,57334.0,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0.000000,NaN,0,...,11,2024,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
13932,57334.0,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0.000000,NaN,0,...,12,2024,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
13933,57334.0,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0.000000,NaN,0,...,1,2025,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
13934,57334.0,BAG BOPP PRINTED W/SIDED SEAL 370X107MM,Non Food,Consumables,Export_Re-Export,LESTER SEOW,NaN,0.000000,NaN,0,...,2,2025,0.0,0.0,0.0,NaN,0,0,1,2024-10-21


In [132]:
sku_demand['variability'] = np.round(sku_demand['variability'], 2)
filter_sku = pd.read_csv('./DATA/INPUT_DATA/filter_BIN_.csv')
c = filter_sku['Bin - Rev'].isin(['A', 'B'])

In [133]:
sku_list_ab_category = filter_sku[c]['MATERIAL'].unique()
len(sku_list_ab_category)

859

In [134]:
total_sku=set(sku_demand['sku'].unique())
a_b_sku=set(sku_list_ab_category)
len(a_b_sku)
non_a_b_sku=total_sku-a_b_sku
len(non_a_b_sku),len(a_b_sku),len(non_a_b_sku)+len(a_b_sku)

(8530, 859, 9389)

In [144]:
a_b_sku_demand=sku_demand[sku_demand['sku'].isin(list(a_b_sku))].reset_index(drop=True)
col__=['rank']+list(a_b_sku_demand.columns)
print(col__)
a_b_sku_demand['rank']=a_b_sku_demand['sku'].rank(method='dense')
a_b_sku_demand.sort_values(['rank'],inplace=True)
a_b_sku_demand=a_b_sku_demand[col__]
max_rank=a_b_sku_demand['rank'].max()
print(max_rank)
non_a_b_sku_demand=sku_demand[sku_demand['sku'].isin(list(non_a_b_sku))].reset_index(drop=True)
non_a_b_sku_demand['rank']=non_a_b_sku_demand['sku'].rank(method='dense')
non_a_b_sku_demand=non_a_b_sku_demand[col__]
non_a_b_sku_demand.sort_values(['rank'],inplace=True)
non_a_b_sku_demand['rank']=non_a_b_sku_demand['rank']+max_rank
final_file=pd.concat([a_b_sku_demand,non_a_b_sku_demand],axis=0).reset_index(drop=True)

['rank', 'sku', 'product_name', 'h1', 'h2', 'channel', 'mrp_purchaser', 'desired_service_level', 'variability', 'three_month_accuracy', 'revision', 'contract_sales', 'final_forecast', 'mape', 'month', 'year', 'statistical_forecast', 'last_year_demand', 'actuals', 'safety_stock', 'statistical', 'total', 'client_id', 'last_updated_at']
817.0


In [145]:
sku_other_details = pd.read_csv('./DATA/INPUT_DATA/PhoonHuat Service Level and Lead times 20240911.csv')
sku_other_details = sku_other_details[["MATERIAL", "SALES_BIN", "CUST_BIN", "SO_BIN"]].drop_duplicates(
    subset=['MATERIAL']
)

In [147]:
sku_other_details.rename(
    columns={"MATERIAL": "sku", "SALES_BIN": "sales_bin", "CUST_BIN": "cust_bin", "SO_BIN": "so_bin"}, inplace=True)
sku_other_details = sku_other_details[['sku', 'sales_bin', 'cust_bin', 'so_bin']]
final_file = pd.merge(final_file, sku_other_details, on=['sku'], how='left')
final_file.to_clipboard(index=False)
# sku_demand_page["statistical"] = 1
# sku_demand_page.to_excel(writer, sheet_name=sheet_name + "SKU_DEMAND_PAGE", index=False)

# writer.close()


In [119]:
a_b_sku_demand

,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,revision,...,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at,rank
64,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,0,...,2024,130.0,120.0,136.0,393.0,1,0,1,2024-10-21,1.0
65,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,0.136512,0,...,2024,148.0,169.0,164.0,393.0,1,0,1,2024-10-21,1.0
66,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,0,...,2024,133.0,134.0,NaN,393.0,1,0,1,2024-10-21,1.0
67,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,0,...,2024,141.0,131.0,NaN,393.0,1,0,1,2024-10-21,1.0
68,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,0,...,2024,135.0,136.0,NaN,393.0,1,0,1,2024-10-21,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177154,202900.0,FRZ SHORT RIB SLICE 2.5MM,Meat,Beef,B2B,None,NaN,0.10,NaN,0,...,2024,603.0,NaN,NaN,NaN,1,0,1,2024-10-21,817.0
177155,202900.0,FRZ SHORT RIB SLICE 2.5MM,Meat,Beef,B2B,None,NaN,0.10,NaN,0,...,2024,603.0,NaN,NaN,NaN,1,0,1,2024-10-21,817.0
177156,202900.0,FRZ SHORT RIB SLICE 2.5MM,Meat,Beef,B2B,None,NaN,0.10,NaN,0,...,2025,603.0,NaN,NaN,NaN,1,0,1,2024-10-21,817.0
177157,202900.0,FRZ SHORT RIB SLICE 2.5MM,Meat,Beef,B2B,None,NaN,0.10,NaN,0,...,2025,603.0,NaN,NaN,NaN,1,0,1,2024-10-21,817.0


In [116]:
a_b_sku_demand=sku_demand[sku_demand['sku'].isin(sku_list_ab_category)]

# a_b_sku_demand=a_b_sku_demand.sort_values(['sku','channel']).reset_index(drop=True).reset_index(names='rank')
# a_b_sku_demand

In [79]:
a_b_sku_demand = sku_demand[sku_demand['sku'].isin(sku_list_ab_category)]
rank_ = pd.DataFrame()
rank_['sku'] = a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + 1
a_b_sku_demand = pd.merge(rank_, a_b_sku_demand, on=['sku'], how='left')

non_a_b_sku_demand = sku_demand[~sku_demand['sku'].isin(sku_list_ab_category)]

ranks = rank_.shape[0]

rank_ = pd.DataFrame()
rank_['sku'] = non_a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + (ranks + 1)

In [82]:
non_a_b_sku_demand = pd.merge(rank_, non_a_b_sku_demand, on=['sku'], how='left')

sku_demand_page_1 = pd.concat([a_b_sku_demand, non_a_b_sku_demand], axis=0)
sku_demand_page_1

,rank,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,...,month,year,statistical_forecast,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at
0,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,8,2024,130.0,120.0,136.0,393.0,1,0,1,2024-10-21
1,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,0.136512,...,9,2024,148.0,169.0,164.0,393.0,1,0,1,2024-10-21
2,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,10,2024,133.0,134.0,NaN,393.0,1,0,1,2024-10-21
3,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,11,2024,141.0,131.0,NaN,393.0,1,0,1,2024-10-21
4,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,12,2024,135.0,136.0,NaN,393.0,1,0,1,2024-10-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168468,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,11,2024,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
168469,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,12,2024,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
168470,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,1,2025,0.0,0.0,0.0,NaN,0,0,1,2024-10-21
168471,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,2,2025,0.0,0.0,0.0,NaN,0,0,1,2024-10-21


In [85]:
# Adding rank section end

# Changing accuracy >>
sku_demand_page_1['three_month_accuracy'] = 1 - sku_demand_page_1['three_month_accuracy']
sku_demand_page_1['three_month_accuracy'] = np.where(sku_demand_page_1['three_month_accuracy'] < 0,
                                                   0, sku_demand_page_1['three_month_accuracy'])

sku_demand_page_1['mape'] = 1 - sku_demand_page_1['mape']
sku_demand_page_1['mape'] = np.where(sku_demand_page_1['mape'] < 0,
                                   0, sku_demand_page_1['mape'])
sku_demand_page_1['mape'] = np.round(sku_demand_page_1['mape'] * 100)

In [86]:
sku_other_details = pd.read_csv('./DATA/INPUT_DATA/PhoonHuat Service Level and Lead times 20240911.csv')
sku_other_details = sku_other_details[["MATERIAL", "SALES_BIN", "CUST_BIN", "SO_BIN"]].drop_duplicates(
    subset=['MATERIAL']
)

In [89]:
sku_other_details.rename(
    columns={"MATERIAL": "sku", "SALES_BIN": "sales_bin", "CUST_BIN": "cust_bin", "SO_BIN": "so_bin"}, inplace=True)
sku_other_details = sku_other_details[['sku', 'sales_bin', 'cust_bin', 'so_bin']]
sku_demand_page_1 = pd.merge(sku_demand_page_1, sku_other_details, on=['sku'], how='left')

# sku_demand_page["statistical"] = 1
# sku_demand_page.to_excel(writer, sheet_name=sheet_name + "SKU_DEMAND_PAGE", index=False)

# writer.close()


In [90]:
sku_demand_page_1

,rank,sku,product_name,h1,h2,channel,mrp_purchaser,desired_service_level,variability,three_month_accuracy,...,last_year_demand,actuals,safety_stock,statistical,total,client_id,last_updated_at,sales_bin,cust_bin,so_bin
0,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,120.0,136.0,393.0,1,0,1,2024-10-21,B,M,Y
1,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,0.136512,...,169.0,164.0,393.0,1,0,1,2024-10-21,B,M,Y
2,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,134.0,NaN,393.0,1,0,1,2024-10-21,B,M,Y
3,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,131.0,NaN,393.0,1,0,1,2024-10-21,B,M,Y
4,1,105.0,BAKING POWDER REDMAN 18KG,Bakery,Functional,B2B,JASYLN,0.95,0.61,NaN,...,136.0,NaN,393.0,1,0,1,2024-10-21,B,M,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191612,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,0.0,0.0,NaN,0,0,1,2024-10-21,NaN,NaN,NaN
191613,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,0.0,0.0,NaN,0,0,1,2024-10-21,NaN,NaN,NaN
191614,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,0.0,0.0,NaN,0,0,1,2024-10-21,NaN,NaN,NaN
191615,9347,680110.0,DETERGENT CLASS B 5LTR,PHD,Non Food,Export_Re-Export,LESTER SEOW,NaN,0.00,NaN,...,0.0,0.0,NaN,0,0,1,2024-10-21,NaN,NaN,NaN


In [81]:
# a_b_sku_demand['sku'].nunique()

817

In [ ]:


a_b_sku_demand = sku_demand[sku_demand['sku'].isin(sku_list_ab_category)]
rank_ = pd.DataFrame()
rank_['sku'] = a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + 1
a_b_sku_demand = pd.merge(rank_, a_b_sku_demand, on=['sku'], how='left')

non_a_b_sku_demand = sku_demand[~sku_demand['sku'].isin(sku_list_ab_category)]

ranks = rank_.shape[0]

rank_ = pd.DataFrame()
rank_['sku'] = non_a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + (ranks + 1)

In [81]:
sku_demand['variability'] = np.round(sku_demand['variability'], 2)
filter_sku = pd.read_csv('./DATA/INPUT_DATA/filter_BIN_.csv')
c = filter_sku['Bin - Rev'].isin(['A', 'B'])

In [82]:
sku_list_ab_category = filter_sku[c]['MATERIAL'].unique()
a_b_sku_demand = sku_demand[sku_demand['sku'].isin(sku_list_ab_category)]
rank_ = pd.DataFrame()
rank_['sku'] = a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + 1
a_b_sku_demand = pd.merge(rank_, a_b_sku_demand, on=['sku'], how='left')

non_a_b_sku_demand = sku_demand[~sku_demand['sku'].isin(sku_list_ab_category)]

ranks = rank_.shape[0]

In [65]:
rank_ = pd.DataFrame()
rank_['sku'] = non_a_b_sku_demand['sku'].unique()
rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
rank_.rename(columns={"index": "rank"}, inplace=True)
rank_['rank'] = rank_['rank'] + (ranks + 1)

In [66]:
non_a_b_sku_demand = pd.merge(rank_, non_a_b_sku_demand, on=['sku'], how='left')

sku_demand_page = pd.concat([a_b_sku_demand, non_a_b_sku_demand], axis=0)

In [67]:
# Adding rank section end

# Changing accuracy >>
sku_demand_page['three_month_accuracy'] = 1 - sku_demand_page['three_month_accuracy']
sku_demand_page['three_month_accuracy'] = np.where(sku_demand_page['three_month_accuracy'] < 0,
                                                   0, sku_demand_page['three_month_accuracy'])

sku_demand_page['mape'] = 1 - sku_demand_page['mape']
sku_demand_page['mape'] = np.where(sku_demand_page['mape'] < 0,
                                   0, sku_demand_page['mape'])
sku_demand_page['mape'] = np.round(sku_demand_page['mape'] * 100)

In [68]:
# # Changing accuracy <<

# # Adding Extra zero sales sku >>

# sample_sku_detail = sku_demand_page.iloc[0, :]
# sku_ = sample_sku_detail['sku']
# channel_ = sample_sku_detail['channel']
# channel_ = sample_sku_detail['channel']
# c = sku_demand_page['sku'] == sku_
# c1 = sku_demand_page['channel'] == channel_
# print(sku_, channel_)
# sample_sku_schema = sku_demand_page[c & c1].drop_duplicates()


# sample_sku_schema['three_month_accuracy'] = np.nan
# sample_sku_schema['statistical'] = 0
# sample_sku_schema['safety_stock'] = np.nan
# sample_sku_schema['last_year_demand'] = 0
# sample_sku_schema['contract_sales'] = 0
# sample_sku_schema['revision'] = 0
# sample_sku_schema['actuals'] = 0
# sample_sku_schema['total'] = 0
# sample_sku_schema['final_forecast'] = 0
# sample_sku_schema['mape'] = np.nan
# sample_sku_schema['variability'] = 0
# sample_sku_schema['statistical_forecast'] = 0

# a = set(sku_demand_page['sku'].unique())
# b = set(sku_details[sku_details['CrossPlantStatus'] == "AC"]['sku'].unique())

# from tqdm import tqdm
# extra_sku = pd.DataFrame()
# for i in tqdm(b - a):
#     c = sku_details['sku'] == i
#     sku_details_ = sku_details[c]
#     dummy_2 = sample_sku_schema.copy()
#     # for j in sku_demand['channel'].unique():
#     for j in ["B2B", "Ecomm", "Retail", "Export_Re-Export"]:
#         dummy_2['sku'] = i
#         dummy_2['channel'] = j
#         dummy_2['product_name'] = sku_details_['PdProductDescription'].values[0]
#         dummy_2['desired_service_level'] = sku_details_['ServiceLevel'].values[0]
#         dummy_2['h1'] = sku_details_['H1'].values[0]
#         dummy_2['h2'] = sku_details_['H2'].values[0]
#     extra_sku = pd.concat([extra_sku, dummy_2], axis=0)

# try:
#     del extra_sku['rank']
# except:
#     pass


# ranks = sku_demand_page['rank'].max() + 1
# if len(b - a) != 0:
#     rank_ = pd.DataFrame()
#     rank_['sku'] = extra_sku['sku'].unique()
#     rank_ = rank_.sort_values('sku').reset_index(drop=True).reset_index()
#     rank_.rename(columns={"index": "rank"}, inplace=True)
#     rank_['rank'] = rank_['rank'] + ranks
#     extra_sku = pd.merge(rank_, extra_sku, on=['sku'], how='left')
#     sku_demand_page = pd.concat([sku_demand_page, extra_sku], axis=0)

# # Adding Extra zero sales sku <<

105.0 B2B


In [76]:
sku_other_details = pd.read_csv('./DATA/INPUT_DATA/PhoonHuat Service Level and Lead times 20240911.csv')
sku_other_details = sku_other_details[["MATERIAL", "SALES_BIN", "CUST_BIN", "SO_BIN"]].drop_duplicates(
    subset=['MATERIAL']
)

In [78]:
sku_other_details.rename(
    columns={"MATERIAL": "sku", "SALES_BIN": "sales_bin", "CUST_BIN": "cust_bin", "SO_BIN": "so_bin"}, inplace=True)
sku_other_details = sku_other_details[['sku', 'sales_bin', 'cust_bin', 'so_bin']]
sku_demand_page = pd.merge(sku_demand_page, sku_other_details, on=['sku'], how='left')

In [79]:
# sku_demand_page["statistical"] = 1
sku_demand_page.to_excel(writer, sheet_name=sheet_name + "SKU_DEMAND_PAGE", index=False)

writer.close()

In [38]:

variability_index = generate_variability_index(input_data_c, current_date)
variability_index

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Bakery B2B
MAKING_DATA _CONTINOUS




  0%|                                                                                         | 0/1415 [00:00<?, ?it/s]

  2%|█▋                                                                             | 30/1415 [00:00<00:04, 284.76it/s]

 11%|████████▍                                                                     | 153/1415 [00:00<00:01, 765.63it/s]

 17%|████████████▉                                                                 | 234/1415 [00:00<00:01, 779.30it/s]

 23%|██████████████████▏                                                           | 330/1415 [00:00<00:01, 807.67it/s]

 29%|██████████████████████▋                                                       | 412/1415 [00:00<00:01, 804.26it/s]

 35%|███████████████████████████▎                                                  | 495/1415 [00:00<00:01, 803.01it/s]

 41%|████████████████████████████████▏                                             | 584/1415 [00:00<00:01, 827.49it/s]

 48%|█████████████████████████

Bakery Ecomm
MAKING_DATA _CONTINOUS




  0%|                                                                                         | 0/1159 [00:00<?, ?it/s]

 10%|███████▋                                                                      | 114/1159 [00:00<00:01, 990.85it/s]

 18%|██████████████▍                                                               | 214/1159 [00:00<00:00, 977.62it/s]

 27%|█████████████████████▍                                                        | 318/1159 [00:00<00:00, 948.42it/s]

 38%|█████████████████████████████▋                                                | 441/1159 [00:00<00:00, 998.13it/s]

 47%|████████████████████████████████████▍                                         | 541/1159 [00:00<00:00, 996.39it/s]

 55%|███████████████████████████████████████████▏                                  | 641/1159 [00:00<00:00, 940.22it/s]

 64%|██████████████████████████████████████████████████▎                           | 747/1159 [00:00<00:00, 957.26it/s]

 75%|█████████████████████████

...

ALL Retail
MAKING_DATA _CONTINOUS




  0%|                                                                                         | 0/6619 [00:00<?, ?it/s]

  1%|▊                                                                              | 64/6619 [00:00<00:10, 627.96it/s]

  2%|█▊                                                                            | 149/6619 [00:00<00:08, 739.42it/s]

  3%|██▋                                                                           | 225/6619 [00:00<00:08, 721.74it/s]

  5%|███▌                                                                          | 298/6619 [00:00<00:08, 714.26it/s]

  6%|████▍                                                                         | 372/6619 [00:00<00:08, 713.78it/s]

  7%|█████▍                                                                        | 457/6619 [00:00<00:08, 747.79it/s]

  8%|██████▎                                                                       | 532/6619 [00:00<00:08, 730.16it/s]

  9%|███████▏                 

ALL ALL
MAKING_DATA _CONTINOUS




  0%|                                                                                         | 0/7604 [00:00<?, ?it/s]

  1%|▋                                                                              | 67/7604 [00:00<00:11, 643.72it/s]

  2%|█▍                                                                            | 142/7604 [00:00<00:10, 697.43it/s]

  3%|██▏                                                                           | 219/7604 [00:00<00:10, 721.56it/s]

  4%|███▏                                                                          | 308/7604 [00:00<00:09, 763.46it/s]

  5%|███▉                                                                          | 385/7604 [00:00<00:09, 732.54it/s]

  6%|████▋                                                                         | 459/7604 [00:00<00:10, 695.68it/s]

  7%|█████▍                                                                        | 530/7604 [00:00<00:10, 699.25it/s]

  8%|██████▏                  

,sku,H1,Channel,Variability,Standard_Dev
0,101.0,Bakery,B2B,0.550142,6.469673
1,102.0,Bakery,B2B,0.407856,3.246537
2,105.0,Bakery,B2B,0.188405,25.193452
3,112.0,Bakery,B2B,0.230704,27.961283
4,116.0,Bakery,B2B,0.832327,23.471614
...,...,...,...,...,...
59633,400145.0,ALL,ALL,NaN,0.000000
59634,670164.0,ALL,ALL,NaN,0.000000
59635,670174.0,ALL,ALL,NaN,0.000000
59636,670175.0,ALL,ALL,NaN,0.000000


In [39]:

variability_data = generate_variability_schema(variability_index)
variability_data.rename(columns={
    "Variability Bucket": "variability_bucket",
    "% Of SKUs": "percentage_of_skus",
    "Channel": "channel",
    "H1": "h1"
}, inplace=True)
variability_data['last_updated_at'] = datetime.now().date()
variability_data.to_csv('./DATA/RESULTS/variability_data.csv')
variability_data.to_excel(writer, sheet_name=sheet_name + "VARIABILITY_INDEX", index=False)


In [40]:
from get_data_mapping import get_data_mapping_sku_demand_page_

In [41]:
data_mapping_sku_demand_page = get_data_mapping_sku_demand_page_(sku_demand_page)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 4823.81it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 734.01it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 553.56it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 685.26it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

100%|███████████████████████████

In [ ]:
c = variability_index['H1'] != 'ALL'
c1 = variability_index['Channel'] == 'ALL'
# std_deviation_details = variability_index[c & c1][['sku', 'Standard_Dev', 'Variability']].drop_duplicates()
# VAIRA NEW CODE >>
input_data__c = input_data_c.copy()
input_data__c['TOTAL_QTY_BASEUOM_SUM'] = input_data__c['TOTAL_QTY_BASEUOM_SUM'].fillna(0)
all_channel_input_data = input_data__c.groupby(['sku', 'year_month'], as_index=False)[['TOTAL_QTY_BASEUOM_SUM']].sum()
all_channel_input_data["date"] = pd.to_datetime(all_channel_input_data['year_month'] + "-01")
c = all_channel_input_data['date'] <= current_date
c1 = all_channel_input_data['date'] > current_date - pd.DateOffset(months=24)
all_channel_input_data = all_channel_input_data[c & c1]

std__ = all_channel_input_data.groupby("sku", as_index=False).agg({'TOTAL_QTY_BASEUOM_SUM': ['mean', 'std']})
std__.columns = ['sku', 'Mean', 'Standard_Dev']
std__["Variability"] = std__["Standard_Dev"] / std__["Mean"]
std_deviation_details = std__.copy()
std_deviation_details.to_csv('./DATA/RESULTS/std_std.csv')

In [ ]:

sku_demand_page = pd.merge(sku_demand_page, std_deviation_details, on=['sku'], how='left')
sku_demand_page = pd.merge(sku_demand_page, input_data_c[['sku', 'year_month', 'Channel', "last_year_demand"]],
                           on=['sku', 'year_month', 'Channel'], how='left')

input_data_c = pd.merge(input_data_c, std_deviation_details, on=['sku'], how='left')

std_deviation_details = pd.merge(std_deviation_details, sku_details[['sku', 'LeadTime', 'ServiceLevel_value']]
                                 , on=['sku'], how='left')

input_data_c = pd.merge(input_data_c.drop('H1', axis=1), sku_details, on=['sku'], how='left')

# OCT 24 new changes avg of previous 12 x 1.2 instead of standard deviation >> Start

c1 = input_data_c['date'] > (current_date - pd.DateOffset(months=12))
c2 = input_data_c['date'] <= current_date
last_one_year_data = input_data_c[c1 & c2]
last_one_year_data = last_one_year_data.groupby(['sku', 'year_month'], as_index=False)['TOTAL_QTY_BASEUOM_SUM'].sum()
last_one_year_data = last_one_year_data.groupby('sku', as_index=False)['TOTAL_QTY_BASEUOM_SUM'].mean()
last_one_year_data['TOTAL_QTY_BASEUOM_SUM_AVG_1.2'] = last_one_year_data['TOTAL_QTY_BASEUOM_SUM'] * 1.2
last_one_year_data.drop('TOTAL_QTY_BASEUOM_SUM', axis=1, inplace=True)
input_data_c = pd.merge(input_data_c, last_one_year_data, on=['sku'], how='left')
std_deviation_details = pd.merge(std_deviation_details, last_one_year_data, on=['sku'], how='left')

# OLD method Std deviation calculation
std_deviation_details['Safety Stock'] = np.sqrt(np.ceil(std_deviation_details['LeadTime'].astype(float) / 30)) * \
                                        std_deviation_details[
                                            'ServiceLevel_value'].astype(float) * (
                                            std_deviation_details['Standard_Dev']) * 1.2


In [ ]:
# OLD METHOD SAFETY STOCK CALCULATION
input_data_c['Safety Stock'] = np.sqrt(np.ceil(sku_demand_page['LeadTime'].astype(float) / 30)) * input_data_c[
    'ServiceLevel_value'].astype(float) * (input_data_c['Standard_Dev']) * 1.2


In [ ]:

input_data_c = pd.merge(input_data_c, forecast_history_data, on=['year_month', 'sku', 'Channel'], how='left')

sku_demand_page['Revision_Sales'] = 0
sku_demand_page['Contract_Sales'] = 0
print("Schema Data started colected")
print(data_mapping_sku_demand_page.shape)
print(data_mapping_sku_demand_page.head())
c = data_mapping_sku_demand_page['Statistical'] == "1"
data_mapping_sku_demand_page = data_mapping_sku_demand_page[c]
print(data_mapping_sku_demand_page.shape, 'PRINT 1')
sku_details['sts'] = 1
data_mapping_sku_demand_page = pd.merge(data_mapping_sku_demand_page,
                                        sku_details[['H1', 'sts']].drop_duplicates(),
                                        on=['H1'], how='left')
c = data_mapping_sku_demand_page['sts'].notnull()
data_mapping_sku_demand_page = data_mapping_sku_demand_page[c]
print(data_mapping_sku_demand_page.shape, "Print2")

data_mapping_sku_demand_page.drop('sts', axis=1, inplace=True)

,0,1,2,3,4
sku,101.0,101.0,101.0,101.0,101.0
Channel,B2B,B2B,B2B,B2B,B2B
date,2019-01-01 00:00:00,2019-02-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-05-01 00:00:00
year_month,2019-01,2019-02,2019-03,2019-04,2019-05
TOTAL_QTY_BASEUOM_SUM,9.0,8.0,20.0,5.0,4.0
TOTAL_QTY_BASEUOM_COUNT,6.0,3.0,9.0,5.0,4.0
TOTAL_SALES_SUM,121.5,108.0,264.0,67.5,54.0
KG_SUM,45.0,40.0,100.0,25.0,20.0
QTY_BASEUOM_SUM,9.0,8.0,20.0,5.0,4.0
QTY_BASEUOM_COUNT,6.0,3.0,9.0,5.0,4.0


In [43]:
from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm

from schema_data_rules import get_data_from_rules, get_data_from_rules_1
from utility import generate_condition, load_filters, round_to_100, make_data_continous_


In [50]:
sku_demand_page['Revision_Sales']=0

In [51]:


sku_demand_schema = get_sku_demand_page_schema(sku_demand_page
                                               , data_mapping_sku_demand_page,
                                               sku_details,
                                               input_data_c,
                                               current_date)

  0%|                                                                                          | 0/120 [00:00<?, ?it/s]

H1 Dairy
Channel Export_Re-Export



  0%|                                                                                          | 0/120 [00:00<?, ?it/s]


KeyError: 'final_perdiction'

In [40]:
sku_demand_page

,train_data_end,train_data_start,horizon,sku,year_month,final_perdiction,Channel,forecast_month,NetWeight,H1,...,PdProductDescription,ProductWeightUnit,NetWeight_calculated,ServiceLevel_value,Mean,Standard_Dev,Variability,last_year_demand,Revision_Sales,Contract_Sales
0,2024-09-01,2022-04-01,1,1039,2024-10,3.571836e-102,Export_Re-Export,2024-10,25.0,Dairy,...,BUTTER UNSALTED/FLECHARD 25KG,KG,25.0,1.645,18.708333,7.641302,0.408444,0.0,0,0
1,2024-09-01,2022-04-01,2,1039,2024-11,3.571836e-102,Export_Re-Export,2024-11,25.0,Dairy,...,BUTTER UNSALTED/FLECHARD 25KG,KG,25.0,1.645,18.708333,7.641302,0.408444,0.0,0,0
2,2024-09-01,2022-04-01,3,1039,2024-12,3.571836e-102,Export_Re-Export,2024-12,25.0,Dairy,...,BUTTER UNSALTED/FLECHARD 25KG,KG,25.0,1.645,18.708333,7.641302,0.408444,0.0,0,0
3,2024-09-01,2022-04-01,4,1039,2025-01,3.571836e-102,Export_Re-Export,2025-01,25.0,Dairy,...,BUTTER UNSALTED/FLECHARD 25KG,KG,25.0,1.645,18.708333,7.641302,0.408444,0.0,0,0
4,2024-09-01,2022-04-01,5,1039,2025-02,3.571836e-102,Export_Re-Export,2025-02,25.0,Dairy,...,BUTTER UNSALTED/FLECHARD 25KG,KG,25.0,1.645,18.708333,7.641302,0.408444,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130579,NaN,NaN,2,104956,2024-11,5.000000e-01,Ecomm,2024-11,0.1,Seafood,...,CAVIAR BLACK LUMPFISH 100G/AKI,KG,0.1,0.842,109.125000,98.130930,0.899253,0.0,0,0
130580,NaN,NaN,3,104956,2024-12,5.000000e-01,Ecomm,2024-12,0.1,Seafood,...,CAVIAR BLACK LUMPFISH 100G/AKI,KG,0.1,0.842,109.125000,98.130930,0.899253,0.0,0,0
130581,NaN,NaN,4,104956,2025-01,5.000000e-01,Ecomm,2025-01,0.1,Seafood,...,CAVIAR BLACK LUMPFISH 100G/AKI,KG,0.1,0.842,109.125000,98.130930,0.899253,2.0,0,0
130582,NaN,NaN,5,104956,2025-02,5.000000e-01,Ecomm,2025-02,0.1,Seafood,...,CAVIAR BLACK LUMPFISH 100G/AKI,KG,0.1,0.842,109.125000,98.130930,0.899253,2.0,0,0


In [38]:
def get_sku_demand_page_schema(acc_final_data, data_mapping_sku_demand_page, sku_details, input_data_c, current_date):
    sku_demand_page_schema = pd.DataFrame()
    for i in tqdm(range(data_mapping_sku_demand_page.shape[0])):

        filters = data_mapping_sku_demand_page.iloc[i, :]
        columns_ = ["Statistical", "Total"]
        rules_ = filters.to_frame().loc[columns_, :]
        rules_index = rules_.index
        rules_values = rules_.iloc[:, 0].values
        rules_values = [rules_index[i] for i, j in enumerate(rules_values) if int(j) != 0]
        rules_values = '_'.join(rules_values)
        # print(filters)
        final_data = get_data_from_rules(acc_final_data, rules_values, 'sku_demand_page')
        for j in ['H1', "Channel"]:
            print(j, filters[j])
            c = generate_condition(final_data, j, filters[j])
            final_data = final_data[c].reset_index(drop=True)
        if final_data.shape[0] == 0:
            continue

        for sku_ in tqdm(final_data['sku'].unique()):
            c = final_data['sku'] == sku_
            t = final_data[c]
            c2 = input_data_c['sku'] == sku_
            c1 = input_data_c['Channel'] == filters['Channel']
            input_ = input_data_c[c1 & c2]
            input_['mape'] = np.abs(input_['final_perdiction'] - input_['QTY_BASEUOM_SUM']) / input_['QTY_BASEUOM_SUM']
            input_['notinf'] = np.where(np.isinf(input_['mape']), 0, 1)
            input_['mape'] = np.where(np.isinf(input_['mape']), 0, input_['mape'])
            input_['3_mape'] = input_['mape'].rolling(3).sum()
            input_['3_notinf'] = input_['notinf'].rolling(3).sum()
            input_['3_months_acc'] = input_['3_mape'] / input_['3_notinf']
            top_df = pd.DataFrame()
            top_df['year_month'] = pd.date_range(current_date - pd.DateOffset(months=2), current_date,
                                                 freq="M") + pd.DateOffset(days=1)
            top_df['year_month'] = top_df['year_month'].astype(str).str[:-3]
            top_df['sku'] = sku_
            top_df['Channel'] = filters["Channel"]
            top_df = pd.merge(top_df, input_[
                ['sku', 'year_month', 'Channel', "last_year_demand", 'TOTAL_QTY_BASEUOM_SUM', 'QTY_BASEUOM_SUM',
                 'Safety Stock',
                 'Variability',
                 'final_perdiction', '3_months_acc', 'contract_sales']], on=['sku', 'year_month', 'Channel'],
                              how='left')
            top_df = pd.merge(top_df, sku_details, on=['sku'], how='left')
            top_df['Variability'] = t['Variability'].max()
            top_df = pd.concat([top_df, t], axis=0).reset_index(drop=True)
            top_df['Statistical'] = filters['Statistical']
            top_df['Total'] = filters['Total']
            top_df['Safety Stock'] = top_df['Safety Stock'].max()
            # display(top_df)

            top_df['Channel'] = filters['Channel']
            top_df['H1'] = filters['H1']
            top_df['H2'] = filters['H2']
            top_df['MrpPurchaser'] = filters['MrpPurchaser']

            sku_demand_page_schema = pd.concat([sku_demand_page_schema, top_df], axis=0)
            print(sku_demand_page_schema.shape)

    return sku_demand_page_schema


In [ ]:
def get_sku_demand_page_schema_(acc_final_data, data_mapping_sku_demand_page, sku_details, input_data_c, current_date):
    sku_demand_page_schema = pd.DataFrame()
    for i in tqdm(range(data_mapping_sku_demand_page.shape[0])):

        filters = data_mapping_sku_demand_page.iloc[i, :]
        columns_ = ["Statistical", "Total"]
        rules_ = filters.to_frame().loc[columns_, :]
        rules_index = rules_.index
        rules_values = rules_.iloc[:, 0].values
        rules_values = [rules_index[i] for i, j in enumerate(rules_values) if int(j) != 0]
        rules_values = '_'.join(rules_values)
        # print(filters)
        final_data = get_data_from_rules(acc_final_data, rules_values, 'sku_demand_page')
        for j in ['H1', "Channel", 'MrpPurchaser', 'H2']:
            print(j, filters[j])
            c = generate_condition(final_data, j, filters[j])
            final_data = final_data[c].reset_index(drop=True)
        # if final_data.shape[0] == 0:
        #     continue

    #     for sku_ in tqdm(final_data['sku'].unique()):
    #         c = final_data['sku'] == sku_
    #         t = final_data[c]
    #         c2 = input_data_c['sku'] == sku_
    #         c1 = input_data_c['Channel'] == filters['Channel']
    #         input_ = input_data_c[c1 & c2]
    #         input_['mape'] = np.abs(input_['final_perdiction'] - input_['QTY_BASEUOM_SUM']) / input_['QTY_BASEUOM_SUM']
    #         input_['notinf'] = np.where(np.isinf(input_['mape']), 0, 1)
    #         input_['mape'] = np.where(np.isinf(input_['mape']), 0, input_['mape'])
    #         input_['3_mape'] = input_['mape'].rolling(3).sum()
    #         input_['3_notinf'] = input_['notinf'].rolling(3).sum()
    #         input_['3_months_acc'] = input_['3_mape'] / input_['3_notinf']
    #         top_df = pd.DataFrame()
    #         top_df['year_month'] = pd.date_range(current_date - pd.DateOffset(months=2), current_date,
    #                                              freq="M") + pd.DateOffset(days=1)
    #         top_df['year_month'] = top_df['year_month'].astype(str).str[:-3]
    #         top_df['sku'] = sku_
    #         top_df['Channel'] = filters["Channel"]
    #         top_df = pd.merge(top_df, input_[
    #             ['sku', 'year_month', 'Channel', "last_year_demand", 'TOTAL_QTY_BASEUOM_SUM', 'QTY_BASEUOM_SUM',
    #              'Safety Stock',
    #              'Variability',
    #              'final_perdiction', '3_months_acc', 'contract_sales']], on=['sku', 'year_month', 'Channel'],
    #                           how='left')
    #         top_df = pd.merge(top_df, sku_details, on=['sku'], how='left')
    #         top_df['Variability'] = t['Variability'].max()
    #         top_df = pd.concat([top_df, t], axis=0).reset_index(drop=True)
    #         top_df['Statistical'] = filters['Statistical']
    #         top_df['Total'] = filters['Total']
    #         top_df['Safety Stock'] = top_df['Safety Stock'].max()
    #         # display(top_df)

    #         top_df['Channel'] = filters['Channel']
    #         top_df['H1'] = filters['H1']
    #         top_df['H2'] = filters['H2']
    #         top_df['MrpPurchaser'] = filters['MrpPurchaser']

    #         sku_demand_page_schema = pd.concat([sku_demand_page_schema, top_df], axis=0)
    #         print(sku_demand_page_schema.shape)

    # return sku_demand_page_schema



In [75]:
data_mapping_sku_demand_page

,H1,Channel,Statistical,Total
0,Dairy,Export_Re-Export,1,0
1,Dairy,Export_Re-Export,0,1
2,Dairy,Retail,1,0
3,Dairy,Retail,0,1
4,Dairy,Ecomm,1,0
...,...,...,...,...
115,Seafood,Retail,0,1
116,Seafood,Ecomm,1,0
117,Seafood,Ecomm,0,1
118,Seafood,B2B,1,0


In [58]:

# variability_index.to_csv("./DATA/RESULTS/varia_index.csv")
print("data map over for sku demand page")
c = variability_index['H1'] != 'ALL'
c1 = variability_index['Channel'] == 'ALL'
# std_deviation_details = variability_index[c & c1][['sku', 'Standard_Dev', 'Variability']].drop_duplicates()
# VAIRA NEW CODE >>
input_data__c = input_data_c.copy()
input_data__c['TOTAL_QTY_BASEUOM_SUM'] = input_data__c['TOTAL_QTY_BASEUOM_SUM'].fillna(0)
all_channel_input_data = input_data__c.groupby(['sku', 'year_month'], as_index=False)[['TOTAL_QTY_BASEUOM_SUM']].sum()
all_channel_input_data["date"] = pd.to_datetime(all_channel_input_data['year_month'] + "-01")
c = all_channel_input_data['date'] <= current_date
c1 = all_channel_input_data['date'] > current_date - pd.DateOffset(months=24)
all_channel_input_data = all_channel_input_data[c & c1]

std__ = all_channel_input_data.groupby("sku", as_index=False).agg({'TOTAL_QTY_BASEUOM_SUM': ['mean', 'std']})
std__.columns = ['sku', 'Mean', 'Standard_Dev']
std__["Variability"] = std__["Standard_Dev"] / std__["Mean"]
std_deviation_details = std__.copy()
std_deviation_details.to_csv('./DATA/RESULTS/std_std.csv')


In [67]:
i='Non Food'
j="EXPOrT"
h2='Some Non Food'

In [71]:
b = np.array([[1, 0], [0, 1]])
a = np.array([[i] * 2, [j] * 2]).T
pd.DataFrame(np.hstack([a, b]))

,0,1,2,3
0,Non Food,EXPOrT,1,0
1,Non Food,EXPOrT,0,1


In [73]:
def get_data_mapping_sku_demand_page_(final_file_schema_1):
    # Main datA Mapping
    data_mapping_1 = pd.DataFrame()
    categories = list(final_file_schema_1['H1'].unique())
    channels = list(final_file_schema_1['Channel'].unique())
    mrp_purchaser = list(final_file_schema_1['MrpPurchaser'].unique())
    h2_list = list(final_file_schema_1['H2'].unique())
    for i in tqdm(categories):
        for j in tqdm(channels):
                    #         print(i,j)
            b = np.array([[1, 0], [0, 1]])
            a = np.array([[i] * 2, [j] * 2,]).T
            combination = pd.DataFrame(np.hstack([a, b]),
                                       columns=['H1', 'Channel', "Statistical", "Total"])
            data_mapping_1 = pd.concat([data_mapping_1, combination], axis=0)
    data_mapping_1.reset_index(drop=True, inplace=True)
    return data_mapping_1


In [53]:
variability_index_.equals(variability_index)

True

In [37]:
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm

from schema_data_rules import get_data_from_rules, get_data_from_rules_1
from utility import generate_condition, load_filters, round_to_100, make_data_continous_

In [26]:
# inpu

In [35]:
from IPython.display import clear_output

In [41]:
def generate_variability_index_(input_data_c, current_date):
    dict_ = {
        "sku": [],
        "H1": [],
        "Channel": [],
        "Variability": [],
        "Standard_Dev": []
    }
    for cat in tqdm(list(input_data_c['H1'].unique()) + ['ALL']):
        #     print(cat)
        for chnl in tqdm(list(input_data_c['Channel'].unique()) + ['ALL']):
            print(cat, chnl)
            c = input_data_c['H1'] == cat
            c = generate_condition(input_data_c, "H1", cat)
            c1 = generate_condition(input_data_c, "Channel", chnl)
            input_data_ = input_data_c[c & c1]
            date_to_filter_data = current_date - pd.DateOffset(months=24)
            date_to_filter_data

            input_data_["date"] = pd.to_datetime(input_data_['year_month'] + "-01")
            c = input_data_['date'] >= date_to_filter_data
            c1 = input_data_['date'] <= current_date
            input_data_ = input_data_[c & c1]

            print(input_data_['date'].max())
            print(input_data_['date'].min())

            input_data_ = input_data_.groupby(['sku', 'year_month'])['TOTAL_QTY_BASEUOM_SUM'].sum().reset_index()
            display(input_data_.head())

        #     return input_data_ 
        # clear_output()
         
            if input_data_.shape[0] == 0:
                continue
            print("MAKING_DATA _CONTINOUS")
            input_data = input_data_.copy()
            # input_data = make_data_continous_(input_data_, 'sku', current_date, 6)
    #         input_data['TOTAL_QTY_BASEUOM_SUM'].fillna(0, inplace=True)
    #         for sku in tqdm(input_data_['sku'].unique()):
    #             dict_['sku'].append(sku)
    #             dict_['H1'].append(cat)
    #             dict_['Channel'].append(chnl)
    #             c = input_data_['sku'] == sku
    #             #             print(input_data[c].shape)
    #             variability = input_data_[c]['TOTAL_QTY_BASEUOM_SUM'].std() / input_data_[c][
    #                 'TOTAL_QTY_BASEUOM_SUM'].mean()
    #             # print(variability, 'printing variability')
    #             dict_['Variability'].append(variability)
    #             dict_['Standard_Dev'].append(input_data_[c]['TOTAL_QTY_BASEUOM_SUM'].std())
    # return pd.DataFrame(dict_)


In [51]:
def generate_variability_index_(input_data_c, current_date):
    dict_ = {
        "sku": [],
        "H1": [],
        "Channel": [],
        "Variability": [],
        "Standard_Dev": []
    }
    for cat in tqdm(list(input_data_c['H1'].unique()) + ['ALL']):
        #     print(cat)
        for chnl in tqdm(list(input_data_c['Channel'].unique()) + ['ALL']):
            print(cat, chnl)
            c = input_data_c['H1'] == cat
            c = generate_condition(input_data_c, "H1", cat)
            c1 = generate_condition(input_data_c, "Channel", chnl)
            input_data_ = input_data_c[c & c1]
            date_to_filter_data = current_date - pd.DateOffset(months=24)
            date_to_filter_data

            input_data_["date"] = pd.to_datetime(input_data_['year_month'] + "-01")
            c = input_data_['date'] >= date_to_filter_data
            c1 = input_data_['date'] <= current_date
            input_data_ = input_data_[c & c1]
            # groupby_col_index=["H1","Channel"]
            # groupby_column=[groupby_col_index[i] for i,j  in enumerate([cat,chnl]) if j.lower() !="all"]+['year_month','sku']
            # print(groupby_column)
            input_data_ = input_data_.groupby(['sku', 'year_month'])['TOTAL_QTY_BASEUOM_SUM'].sum().reset_index()
            if input_data_.shape[0] == 0:
                continue
            print("MAKING_DATA _CONTINOUS")
            # input_data = make_data_continous_(input_data_, 'sku', current_date, 6)
            input_data_['date']=pd.to_datetime(input_data_['year_month']+"-01")
            input_data_.sort_values(['sku','date'],inplace=True)
            input_data=input_data_.copy()
            input_data['TOTAL_QTY_BASEUOM_SUM'].fillna(0, inplace=True)
            for sku in tqdm(input_data_['sku'].unique()):
                dict_['sku'].append(sku)
                dict_['H1'].append(cat)
                dict_['Channel'].append(chnl)
                c = input_data_['sku'] == sku
                #             print(input_data[c].shape)
                variability = input_data_[c]['TOTAL_QTY_BASEUOM_SUM'].std() / input_data_[c][
                    'TOTAL_QTY_BASEUOM_SUM'].mean()
                # print(variability, 'printing variability')
                dict_['Variability'].append(variability)
                dict_['Standard_Dev'].append(input_data_[c]['TOTAL_QTY_BASEUOM_SUM'].std())
    return pd.DataFrame(dict_)


In [26]:
from tqdm import tqdm

In [27]:
for i in tqdm(range(sku_channel_history.shape[0])):
    # print(i)
    # print(sku_details_
    d = pd.DataFrame()
    d['year_month'] = pd.Series(
        pd.date_range(current_date - pd.DateOffset(months=6), current_date, freq="M")) + pd.DateOffset(days=1)
    d['year_month'] = d['year_month'].astype(str).str[:-3]
    d['sku'] = sku_details_['sku']
    d['Channel'] = sku_details_['Channel']
    # display(d)
    sku_channel_history_data = pd.concat([sku_channel_history_data, d], axis=0)

100%|███████████████████████████████████████████████████████████████████████████| 37719/37719 [05:01<00:00, 125.11it/s]


In [44]:
# sku_channel_history_data[['sku','Channel']].drop_duplicates()
sku_channel_history_data.shape

(252462, 3)

In [47]:
# forecast_history_data__ = pd.merge(sku_channel_history_data, forecast_history_data, on=['sku', 'year_month', 'Channel'],
#                                  how='left')

In [48]:
# forecast_history_data__.to_clipboard(index=False)

In [43]:
sku_channel_history.shape

(37719, 3)

In [35]:
history_date_range=(pd.date_range(current_date - pd.DateOffset(months=6), current_date, freq="M") + pd.DateOffset(days=1)).astype(str).str[:-3]

In [37]:
sku_channel_history['grp']=1
sku_channel_history.head()

,sku,Channel,grp
0,1039,Export_Re-Export,1
1,1087,Export_Re-Export,1
2,1088,Export_Re-Export,1
3,1095,Export_Re-Export,1
4,1917,Export_Re-Export,1


In [41]:
dummy_data=pd.DataFrame({'grp':[1],'year_month':[history_date_range]})
sku_channel_history_data_=pd.merge(sku_channel_history,dummy_data,on=['grp'],how='left')
sku_channel_history_data_.explode('year_month')

,sku,Channel,grp,year_month
0,1039,Export_Re-Export,1,2024-04
0,1039,Export_Re-Export,1,2024-05
0,1039,Export_Re-Export,1,2024-06
0,1039,Export_Re-Export,1,2024-07
0,1039,Export_Re-Export,1,2024-08
...,...,...,...,...
21702,111259,Export_Re-Export,1,2024-05
21702,111259,Export_Re-Export,1,2024-06
21702,111259,Export_Re-Export,1,2024-07
21702,111259,Export_Re-Export,1,2024-08


In [71]:
max_horizon=6

In [73]:
str(((current_date+pd.DateOffset(months=1))+pd.DateOffset(months=max_horizon-1)).date())[:-3]

'2025-03'

In [ ]:
# METHOD 1
def make_input_data_continous_new(input_data,current_date):
    # Setting Date Ranges 
    # current_year_month=str((current_date+pd.DateOffset(months=1)).date())[:-3]
    current_year_month=str(((current_date+pd.DateOffset(months=1))+pd.DateOffset(months=max_horizon-1)).date())[:-3]
    date_ranges=np.arange('2018-07',current_year_month,dtype='datetime64[M]')
    # Adding the date range into a dummy
    group_dict={'group':[1],'date':[date_ranges]}
    group_dict=pd.DataFrame(group_dict)
    # GEtting unique MATERIAL Channel & Category
    unique_skus=input_data[['MATERIAL','CHNL_NAME','H1']]
    input_data_cc=pd.DataFrame()
    input_data_cc=unique_skus.drop_duplicates().copy()
    input_data_cc['group']=1
    input_data_cc=pd.merge(input_data_cc,group_dict,on=['group'],how='left')
    input_data_cc=input_data_cc.explode(column='date')
    input_data['date']=pd.to_datetime(input_data['year_month']+"-01")
    min_input_date_=input_data.groupby(['MATERIAL',"CHNL_NAME"],as_index=False)['date'].min().rename(columns={'date':'min_date'})
    input_data_cc=pd.merge(input_data_cc,min_input_date_,on=['MATERIAL','CHNL_NAME'],how='left')
    c=input_data_cc['date']>=input_data_cc['min_date']
    input_data_cc=input_data_cc[c]
    input_data_c=pd.merge(input_data_cc,input_data.drop('H1',axis=1),on=['MATERIAL','date','CHNL_NAME'],how='left')
    input_data_c.head()
    input_data_c['year_month']=input_data_c['date'].astype(str).str[:-3]
    input_data_c.fillna(0,inplace=True)
    input_data_c.drop(['group','min_date'],axis=1,inplace=True)
    return input_data_c

In [111]:
unique_skus=input_data[['MATERIAL','CHNL_NAME','H1']]

In [112]:
group_dict={'group':[1],'date':[date_ranges]}
group_dict=pd.DataFrame(group_dict)
group_dict

,group,date
0,1,"[2018-07, 2018-08, 2018-09, 2018-10, 2018-11, ..."


In [113]:
input_data_cc=pd.DataFrame()
input_data_cc=unique_skus.drop_duplicates().copy()
input_data_cc['group']=1
input_data_cc

,MATERIAL,CHNL_NAME,H1,group
0,101.0,Retail,Bakery,1
1,101.0,B2B,Bakery,1
40,101.0,Export_Re-Export,Bakery,1
56,101.0,Ecomm,Bakery,1
168,102.0,B2B,Bakery,1
...,...,...,...,...
672008,400145.0,Export_Re-Export,Bakery,1
672009,670164.0,B2B,Non Food,1
672014,670174.0,B2B,Non Food,1
672015,670175.0,B2B,Non Food,1


In [114]:
input_data_cc=pd.merge(input_data_cc,group_dict,on=['group'],how='left')
input_data_cc=input_data_cc.explode(column='date')
input_data_cc

,MATERIAL,CHNL_NAME,H1,group,date
0,101.0,Retail,Bakery,1,2018-07-01
0,101.0,Retail,Bakery,1,2018-08-01
0,101.0,Retail,Bakery,1,2018-09-01
0,101.0,Retail,Bakery,1,2018-10-01
0,101.0,Retail,Bakery,1,2018-11-01
...,...,...,...,...,...
22223,670176.0,Export_Re-Export,Non Food,1,2024-05-01
22223,670176.0,Export_Re-Export,Non Food,1,2024-06-01
22223,670176.0,Export_Re-Export,Non Food,1,2024-07-01
22223,670176.0,Export_Re-Export,Non Food,1,2024-08-01


In [115]:
input_data['date']=pd.to_datetime(input_data['year_month']+"-01")

In [116]:
input_data

,MATERIAL,year_month,CHNL_NAME,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM,H1,date
0,101.0,2018-12,Retail,1,1,14.30,5.0,1.0,1.0,14.30,Bakery,2018-12-01
1,101.0,2019-01,B2B,9,6,121.50,45.0,9.0,6.0,121.50,Bakery,2019-01-01
2,101.0,2019-01,Retail,9,8,125.81,45.0,9.0,8.0,125.81,Bakery,2019-01-01
3,101.0,2019-02,B2B,8,3,108.00,40.0,8.0,3.0,108.00,Bakery,2019-02-01
4,101.0,2019-02,Retail,10,8,138.65,50.0,10.0,8.0,138.65,Bakery,2019-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...
672012,670164.0,2021-10,B2B,8,1,44.00,4.0,8.0,1.0,44.00,Non Food,2021-10-01
672013,670164.0,2021-11,B2B,79,2,434.50,39.5,79.0,2.0,434.50,Non Food,2021-11-01
672014,670174.0,2021-12,B2B,200,1,8.00,0.6,200.0,1.0,8.00,Non Food,2021-12-01
672015,670175.0,2021-12,B2B,200,1,10.00,1.0,200.0,1.0,10.00,Non Food,2021-12-01


In [117]:
min_input_date_=input_data.groupby(['MATERIAL',"CHNL_NAME"],as_index=False)['date'].min().rename(columns={'date':'min_date'})

In [118]:
input_data_cc=pd.merge(input_data_cc,min_input_date_,on=['MATERIAL','CHNL_NAME'],how='left')
input_data_cc

,MATERIAL,CHNL_NAME,H1,group,date,min_date
0,101.0,Retail,Bakery,1,2018-07-01,2018-12-01
1,101.0,Retail,Bakery,1,2018-08-01,2018-12-01
2,101.0,Retail,Bakery,1,2018-09-01,2018-12-01
3,101.0,Retail,Bakery,1,2018-10-01,2018-12-01
4,101.0,Retail,Bakery,1,2018-11-01,2018-12-01
...,...,...,...,...,...,...
1666795,670176.0,Export_Re-Export,Non Food,1,2024-05-01,2023-03-01
1666796,670176.0,Export_Re-Export,Non Food,1,2024-06-01,2023-03-01
1666797,670176.0,Export_Re-Export,Non Food,1,2024-07-01,2023-03-01
1666798,670176.0,Export_Re-Export,Non Food,1,2024-08-01,2023-03-01


In [124]:
c=input_data_cc['date']>=input_data_cc['min_date']
input_data_cc=input_data_cc[c]

In [130]:
# input_data_cc=
input_data_c=pd.merge(input_data_cc,input_data.drop('H1',axis=1),on=['MATERIAL','date','CHNL_NAME'],how='left')
input_data_c.head()

,MATERIAL,CHNL_NAME,H1,group,date,min_date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM
0,101.0,Retail,Bakery,1,2018-12-01,2018-12-01,2018-12,1.0,1.0,14.30,5.0,1.0,1.0,14.30
1,101.0,Retail,Bakery,1,2019-01-01,2018-12-01,2019-01,9.0,8.0,125.81,45.0,9.0,8.0,125.81
2,101.0,Retail,Bakery,1,2019-02-01,2018-12-01,2019-02,10.0,8.0,138.65,50.0,10.0,8.0,138.65
3,101.0,Retail,Bakery,1,2019-03-01,2018-12-01,2019-03,10.0,9.0,138.67,50.0,10.0,9.0,138.67
4,101.0,Retail,Bakery,1,2019-04-01,2018-12-01,2019-04,16.0,15.0,220.20,80.0,16.0,15.0,220.20


In [133]:
input_data_c['year_month']=input_data_c['date'].astype(str).str[:-3]

In [136]:
input_data_c.fillna(0,inplace=True)

In [125]:
# # TESTING START
# c=input_data_cc['H1']=="Non Food"
# sample_sku=input_data_cc['MATERIAL'].sample(100).unique()
# c1=input_data_cc['MATERIAL'].isin(sample_sku)
# input_data_cc[c & c1].to_clipboard(index=False)
# # TESTING STOP

In [126]:
# input_data_cc.shape

(1158977, 6)

,MATERIAL,CHNL_NAME,H1,group,date,min_date,year_month,TOTAL_QTY_BASEUOM_SUM,TOTAL_QTY_BASEUOM_COUNT,TOTAL_SALES_SUM,KG_SUM,QTY_BASEUOM_SUM,QTY_BASEUOM_COUNT,SALES_SUM
0,101.0,Retail,Bakery,1,2018-12-01,2018-12-01,2018-12,1.0,1.0,14.30,5.0,1.0,1.0,14.30
1,101.0,Retail,Bakery,1,2019-01-01,2018-12-01,2019-01,9.0,8.0,125.81,45.0,9.0,8.0,125.81
2,101.0,Retail,Bakery,1,2019-02-01,2018-12-01,2019-02,10.0,8.0,138.65,50.0,10.0,8.0,138.65
3,101.0,Retail,Bakery,1,2019-03-01,2018-12-01,2019-03,10.0,9.0,138.67,50.0,10.0,9.0,138.67
4,101.0,Retail,Bakery,1,2019-04-01,2018-12-01,2019-04,16.0,15.0,220.20,80.0,16.0,15.0,220.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158972,670176.0,Export_Re-Export,Non Food,1,2024-05-01,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1158973,670176.0,Export_Re-Export,Non Food,1,2024-06-01,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1158974,670176.0,Export_Re-Export,Non Food,1,2024-07-01,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1158975,670176.0,Export_Re-Export,Non Food,1,2024-08-01,2023-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


7605